doc processing / chunking classes for storing in vector database (folder_path="/data/user_data/vidhij2/medical_db/rag", embeddings="google/muril-base-cased")
( heirarchical and semantic)
trying to preprocess and a few document and test a bit  
reasning based rag models
hybrid chunking ( v.v. long class def) 
making vector store and testing a bit hybrid rag - answeing as well 
Enhanced RAG class for maternal health with multilingual support, stage-aware retrieval, and cross-encoder reranking. [ class for retrieval afer final vector store is done]
testing with different models the enhanced rag class and storing the answers [ mp used this for last results ]

# Storing data for rag database

In [5]:
import pandas as pd

# 1.1: Load the Excel file
df = pd.read_excel("/home/vidhij2/nivi/eSaathi Resources_ANC_PNC_CMU.xlsx", sheet_name="Sheet1")

# 1.2: Assume there’s a column named "resource_link" with PDF or webpage URLs
pairs = df[["Title", "Link"]].dropna(subset=["Link"]).to_dict(orient="records")


In [3]:
df

,Title,Link,Used for what part of MH journey,Document Category
0,WHO recommendations on antenatal care for a po...,https://www.who.int/publications/i/item/978924...,ANC,Clinical Guidelines and Recommendations
1,WHO recommendations on maternal and newborn ca...,https://www.who.int/publications/i/item/978924...,PNC,Clinical Guidelines and Recommendations
2,Maternal Health,https://nhm.gov.in/index1.php?lang=1&level=2&l...,ANC/PNC,Government Policy and Programs
3,IEC Materials (might not be accessible in US),https://nhm.gov.in/index1.php?lang=1&level=3&s...,Audio/Visuals,Visual and Educational Resources
4,MH Guidelines (might not be accessible in US),https://nhm.gov.in/index1.php?lang=1&level=3&s...,ANC/PNC,Clinical Guidelines and Recommendations
...,...,...,...,...
247,Medicover Hospitals,https://www.medicoverhospitals.in/articles/pre...,NaN,NaN
248,Renew Healthcare India,https://renewhealthcare.in/double-the-trouble-...,NaN,NaN
249,Matri Women's Health,https://maitriwoman.com/planning-a-pregnancy-a...,NaN,NaN
250,Motherhood Hospitals,https://www.motherhoodindia.com/pregnancy-afte...,NaN,NaN


In [2]:
import os

os.makedirs("/data/user_data/vidhij2/medical/pdfs", exist_ok=True)
os.makedirs("/data/user_data/vidhij2/medical/html", exist_ok=True)

In [9]:
import os
import re
import pandas as pd
import requests
def sanitize_filename(name: str) -> str:
    """
    Replace or remove characters that are invalid in filenames.
    Here we:
      - Strip leading/trailing whitespace
      - Replace spaces with underscores
      - Remove any characters in <>:"/\\|?*  
    """
    name = name.strip()
    name = name.replace(" ", "_")
    # remove any character not in [a-zA-Z0-9_.-]
    name = re.sub(r'[<>:"/\\|?*]', "", name)
    return name

for item in pairs:
    raw_title = str(item["Title"])
    url       = item["Link"]

    # 2.3.1: sanitize the title so we can safely use it as a filename
    safe_title = sanitize_filename(raw_title)

    try:
        if url.lower().endswith(".pdf"):
            resp = requests.get(url, timeout=15)
            resp.raise_for_status()
            outpath = f"/data/user_data/vidhij2/medical/pdfs/{safe_title}.pdf"
            with open(outpath, "wb") as f:
                f.write(resp.content)
            print(f"Downloaded PDF → {outpath}")

        else:
            resp = requests.get(url, timeout=15)
            resp.raise_for_status()
            outpath = f"/data/user_data/vidhij2/medical/html/{safe_title}.html"
            with open(outpath, "w", encoding="utf-8") as f:
                f.write(resp.text)
            print(f"Saved HTML → {outpath}")

    except Exception as e:
        print(f"⚠️ Failed to fetch “{raw_title}” ({url}): {e}")

Saved HTML → /data/user_data/vidhij2/medical/html/WHO_recommendations_on_antenatal_care_for_a_positive_pregnancy_experience.html
Saved HTML → /data/user_data/vidhij2/medical/html/WHO_recommendations_on_maternal_and_newborn_care_for_a_positive_postnatal_experience.html
Saved HTML → /data/user_data/vidhij2/medical/html/Maternal_Health.html
Saved HTML → /data/user_data/vidhij2/medical/html/IEC_Materials_(might_not_be_accessible_in_US).html
Saved HTML → /data/user_data/vidhij2/medical/html/MH_Guidelines_(might_not_be_accessible_in_US).html
Saved HTML → /data/user_data/vidhij2/medical/html/Digital_Adaptation_Kit_for_Antenatal_Care_Operational_requirements_for_implementing_WHO_recommendations_in_digital_systems.html
Downloaded PDF → /data/user_data/vidhij2/medical/pdfs/Pregnant_now_or_within_the_last_year.pdf
Downloaded PDF → /data/user_data/vidhij2/medical/pdfs/Maternal_nutrition_for_safe_motherhood_(Flipbook).pdf
Downloaded PDF → /data/user_data/vidhij2/medical/pdfs/Maternal_nutrition_for_

# maternal health pipeline ( used this for hybrid rag)

In [1]:
# ─── NEW: helper wrapper ────────────────────────────────────────────────────────
from langchain.embeddings import HuggingFaceEmbeddings

class E5Embeddings(HuggingFaceEmbeddings):
    """
    Adds the 'query: ' / 'passage: ' prefixes that E5‑instruct models expect.
    Everything else (batch‑ing, normalisation, device etc.) is inherited.
    """
    def __init__(self, device: str = "cpu",
                 model_name: str = "intfloat/multilingual-e5-large-instruct",
                 batch_size: int = 32):
        super().__init__(
            model_name=model_name,
            model_kwargs={"device": device},
            encode_kwargs={
                "normalize_embeddings": True,
                "batch_size": batch_size
            },
        )
        self._query_prefix   = "query: "
        self._passage_prefix = "passage: "

    # LangChain will call these two methods internally  ↓
    def embed_query(self, text: str):
        return super().embed_query(self._query_prefix + text)

    def embed_documents(self, texts):
        texts = [self._passage_prefix + t for t in texts]
        return super().embed_documents(texts)
# ────────────────────────────────────────────────────────────────────────────────


In [1]:
import os
import re
import json
import uuid
from typing import List, Dict, Any, Optional, Tuple, Set
from dataclasses import dataclass
from functools import lru_cache
import logging
from tqdm import tqdm

# LangChain imports
from langchain.document_loaders import TextLoader, PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document
from langchain.schema import BaseDocumentTransformer
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler("maternal_health_pipeline.log"),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

class MaternalHealthPipeline:
    """
    Comprehensive pipeline for processing and retrieving maternal health documents.
    This implements a hybrid semantic-hierarchical chunking approach optimized for
    maternal health content and patient queries.
    """
    
    def __init__(
        self,
        chunk_size: int = 2500,
        chunk_overlap: int = 400,
        db_directory: str = "maternal_health_db",
        spacy_model: str = "en_core_web_md",
        use_spacy: bool = True
    ):
        """
        Initialize the maternal health document pipeline.
        
        Args:
            chunk_size: Target size for document chunks
            chunk_overlap: Amount of overlap between chunks
            db_directory: Directory to store vector database
            spacy_model: SpaCy model for NLP processing
            use_spacy: Whether to use SpaCy for enhanced NLP
        """
        self.chunk_size = chunk_size
        self.chunk_overlap = chunk_overlap

        self.db_directory = db_directory
        self.db = None
        
        # Create db directory if it doesn't exist
        os.makedirs(db_directory, exist_ok=True)
        # Initialize embeddings model
        self.embeddings = E5Embeddings(device="cuda")
        # Initialize SpaCy if requested
        self.use_spacy = use_spacy
        if use_spacy:
            try:
                import spacy
                try:
                    self.nlp = spacy.load(spacy_model)
                    logger.info(f"Loaded SpaCy model: {spacy_model}")
                except OSError:
                    logger.info(f"Downloading SpaCy model: {spacy_model}")
                    spacy.cli.download(spacy_model)
                    self.nlp = spacy.load(spacy_model)
            except ImportError:
                logger.warning("SpaCy not available. Some features will be limited.")
                self.use_spacy = False
                self.nlp = None
        else:
            self.nlp = None

        # Check for existing vector store
        index_path = os.path.join(db_directory, "index.faiss")
        if os.path.exists(index_path):
            logger.info(f"Loading existing vector store from {index_path}")
            self.db = FAISS.load_local(
                folder_path=db_directory,
                embeddings=self.embeddings,
                allow_dangerous_deserialization=True
            )
        else:
            self.db = None
        
        # Compile regex patterns
        self._compile_regex_patterns()
        
        # Initialize domain-specific knowledge
        self._initialize_domain_knowledge()
    
    def _compile_regex_patterns(self):
        """Compile all regex patterns used across the pipeline for performance."""
        # Document structure patterns
        self.structure_patterns = {
            # Formal headers in guidelines
            'formal_header': re.compile(r'^(?:[0-9]+\.){1,3}\s+[A-Z]|^(?:CHAPTER|Section|SECTION)\s+[0-9IVX]+', re.MULTILINE),
            
            # Natural language headers
            'nl_header': re.compile(r'^[A-Z][A-Za-z\s]+:$|^[A-Z][A-Za-z\s]+ during [A-Za-z\s]+$', re.MULTILINE),
            
            # Special content markers
            'special_marker': re.compile(r'^(?:NOTE|WARNING|CAUTION|RECOMMENDATION|EVIDENCE|BOX|Table|Figure):?', re.MULTILINE),
            
            # Document boundaries
            'doc_boundary': re.compile(r'(?:\n\s*){3,}|\f|(?:=+\s*){3,}|(?:-+\s*){3,}')
        }
        
        # Content preservation patterns
        self.preservation_patterns = {
            # Lists
            'list': re.compile(r'(?:^[•\-\*√]\s+.+(?:\n|$)){2,}', re.MULTILINE),
            
            # Tables (various formats)
            'table': re.compile(r'(?:\|.+\|.+\|.+\n){2,}|(?:^[\w\s]+\t+[\w\s]+\t+[\w\s]+(?:\n|$)){2,}', re.MULTILINE),
            
            # Warning blocks
            'warning': re.compile(r'(?:Warning|Caution|Alert|WARNING|CAUTION|ALERT):[\s\S]+?(?:\n\n|\Z)', re.MULTILINE),
            
            # Recommendation blocks
            'recommendation': re.compile(r'(?:Recommendation|RECOMMENDATION):[\s\S]+?(?:\n\n|\Z)', re.MULTILINE),
            
            # Numbered steps/procedures
            'steps': re.compile(r'(?:^[0-9]+\.\s+.+(?:\n|$)){2,}', re.MULTILINE)
        }
        
        # Pregnancy stage patterns
        self.stage_patterns = {
            'week': re.compile(r'(?:week|weeks)\s+(\d+(?:-\d+)?)|(\d+)(?:st|nd|rd|th)?\s+week', re.IGNORECASE),
            'month': re.compile(r'(?:month|months)\s+(\d+(?:-\d+)?)|(\d+)(?:st|nd|rd|th)?\s+month', re.IGNORECASE),
            'trimester': re.compile(r'(first|second|third)\s+trimester', re.IGNORECASE),
            'stage': re.compile(r'(preconception|prenatal|antenatal|perinatal|intrapartum|postpartum|postnatal)', re.IGNORECASE)
        }
        
        # Clinical patterns
        self.clinical_patterns = {
            'measurement': re.compile(r'(\d+(?:\.\d+)?)\s*(cm|mm|kg|g|lb|oz|bpm|mmHg|%|mg|mcg|IU)', re.IGNORECASE),
            'dosage': re.compile(r'(\d+(?:\.\d+)?)\s*(mg|mcg|g|ml|IU|units)\s+(\d+(?:-\d+)?)\s*times?(\/day|\/week|daily|weekly)', re.IGNORECASE)
        }
    
    def _initialize_domain_knowledge(self):
        """Initialize comprehensive domain knowledge for maternal health."""
        # Document categories
        self.doc_categories = {
            'clinical': ['guideline', 'protocol', 'recommendation', 'clinical', 'medical', 'treatment', 'diagnosis', 'procedure'],
            'educational': ['education', 'information', 'guide', 'leaflet', 'brochure', 'advice'],
            'nutritional': ['nutrition', 'diet', 'food', 'eating', 'meal', 'vitamin', 'supplement'],
            'behavioral': ['behavior', 'behaviour', 'habit', 'lifestyle', 'exercise', 'activity', 'practice'],
            'emotional': ['emotional', 'mental', 'psychological', 'feeling', 'support', 'stress', 'anxiety', 'depression']
        }
        
        # Pregnancy stages (for improved stage detection)
        self.pregnancy_stages = {
            'preconception': ['planning', 'before pregnancy', 'preconception', 'pre-conception', 'fertility'],
            'first_trimester': ['first trimester', 'early pregnancy', 'weeks 1-12', '1st trimester'],
            'second_trimester': ['second trimester', 'mid pregnancy', 'weeks 13-26', '2nd trimester'],
            'third_trimester': ['third trimester', 'late pregnancy', 'weeks 27-40', '3rd trimester'],
            'labor_delivery': ['labor', 'labour', 'delivery', 'birth', 'childbirth', 'intrapartum'],
            'postpartum': ['postpartum', 'postnatal', 'after birth', 'puerperium', 'fourth trimester', '4th trimester']
        }
        
        # Common maternal health topics
        self.maternal_topics = {
            'fetal_development': ['development', 'growth', 'fetal', 'foetal', 'baby', 'fetus', 'embryo'],
            'maternal_changes': ['mother', 'maternal', 'body', 'changes', 'symptoms', 'physical'],
            'nutrition': ['diet', 'food', 'nutrition', 'eat', 'meal', 'vitamin', 'nutrient', 'folate', 'iron'],
            'warning_signs': ['danger', 'warning', 'risk', 'emergency', 'complication', 'problem'],
            'normal_variations': ['normal', 'common', 'usual', 'typical', 'variant', 'variation', 'expected'],
            'medical_tests': ['test', 'scan', 'ultrasound', 'screening', 'monitor', 'measurement'],
            'delivery': ['birth', 'labor', 'labour', 'delivery', 'contractions', 'childbirth'],
            'postpartum': ['after birth', 'post-delivery', 'postpartum', 'post-natal', 'breastfeeding'],
            'mental_health': ['mental', 'depression', 'anxiety', 'mood', 'stress', 'emotional'],
            'prenatal_care': ['prenatal', 'antenatal', 'checkup', 'visit', 'appointment', 'healthcare']
        }
        
        # Common patient concerns (for enhanced retrieval)
        self.patient_concerns = {
            'pain': ['pain', 'ache', 'discomfort', 'hurt', 'sore', 'cramp'],
            'bleeding': ['bleed', 'blood', 'spotting', 'hemorrhage', 'discharge'],
            'movement': ['movement', 'kick', 'active', 'moving', 'stirring', 'flutter'],
            'nausea': ['nausea', 'vomit', 'sick', 'morning sickness', 'hyperemesis'],
            'sleep': ['sleep', 'insomnia', 'tired', 'fatigue', 'exhausted', 'rest'],
            'emotions': ['emotion', 'mood', 'cry', 'anxiety', 'worry', 'fear', 'depression'],
            'nutrition': ['eat', 'food', 'diet', 'craving', 'appetite', 'weight', 'nutrition']
        }
        
        # Medical entities
        self.medical_entities = {
            'conditions': [
                'preeclampsia', 'eclampsia', 'gestational diabetes', 'anemia', 'anaemia',
                'hypertension', 'nausea', 'vomiting', 'morning sickness', 'hyperemesis',
                'placenta previa', 'preterm labor', 'preterm birth', 'ectopic pregnancy',
                'miscarriage', 'bleeding', 'spotting', 'edema', 'swelling', 'back pain',
                'postpartum depression', 'postpartum hemorrhage', 'mastitis', 'HELLP syndrome',
                'gestational hypertension', 'cholestasis', 'UTI', 'yeast infection'
            ],
            'medications': [
                'iron', 'folate', 'folic acid', 'vitamin', 'supplement', 'prenatal vitamin',
                'calcium', 'magnesium', 'zinc', 'omega-3', 'aspirin', 'antacid', 'insulin',
                'labetalol', 'methyldopa', 'nifedipine', 'paracetamol', 'acetaminophen',
                'ibuprofen', 'metformin', 'heparin', 'progesterone', 'oxytocin', 'epidural'
            ],
            'procedures': [
                'ultrasound', 'scan', 'test', 'screening', 'amniocentesis', 'delivery',
                'c-section', 'cesarean', 'caesarean', 'induction', 'epidural', 'monitoring',
                'glucose test', 'blood test', 'urine test', 'group b strep', 'genetic screening',
                'CVS', 'chorionic villus sampling', 'NIPT', 'NST', 'non-stress test',
                'biophysical profile', 'cervical check', 'membrane sweep'
            ],
            'measurements': [
                'weight', 'blood pressure', 'fundal height', 'bp', 'heart rate', 'fetal heart rate',
                'bpm', 'glucose', 'temperature', 'hcg', 'hemoglobin', 'haemoglobin', 'blood sugar',
                'contractions', 'dilation', 'effacement', 'station', 'bilirubin', 'amniotic fluid index'
            ]
        }
    
    @lru_cache(maxsize=32)
    def _extract_document_metadata(self, text_sample: str, filename: str = None) -> Dict[str, Any]:
        """
        Extract document-level metadata from text sample.
        
        Args:
            text_sample: Representative sample of document text
            filename: Original document filename
            
        Returns:
            Dict containing document metadata
        """
        text_lower = text_sample.lower()
        metadata = {
            "filename": filename,
            "document_categories": []
        }
        
        # Extract document categories
        for category, keywords in self.doc_categories.items():
            if any(keyword in text_lower for keyword in keywords):
                metadata["document_categories"].append(category)
        
        # Determine document type by content and filename patterns
        doc_type = "Maternal Health Document"  # Default
        
        if filename:
            filename_lower = filename.lower()
            if any(term in filename_lower for term in ['guideline', 'guide', 'protocol']):
                if 'anc' in filename_lower or 'prenatal' in filename_lower or 'antenatal' in filename_lower:
                    doc_type = "Antenatal Care Guideline"
                elif 'pnc' in filename_lower or 'postnatal' in filename_lower or 'postpartum' in filename_lower:
                    doc_type = "Postnatal Care Guideline"
                elif 'nutr' in filename_lower or 'diet' in filename_lower or 'food' in filename_lower:
                    doc_type = "Maternal Nutrition Guide"
                elif 'ppd' in filename_lower or 'depress' in filename_lower or 'mental' in filename_lower:
                    doc_type = "Maternal Mental Health Guide"
                else:
                    doc_type = "Clinical Guideline"
        
        # Look for title in content
        title_pattern = re.compile(r'^[A-Z][\w\s:]+(?:Guidelines?|Protocol|Care|Health|Management)', re.MULTILINE)
        title_match = title_pattern.search(text_sample[:2000])
        if title_match:
            metadata["title"] = title_match.group(0).strip()
        else:
            # Try to construct title from filename if available
            if filename:
                base_name = os.path.splitext(os.path.basename(filename))[0]
                # Convert snake_case or kebab-case to Title Case
                base_name = re.sub(r'[_-]', ' ', base_name).title()
                metadata["title"] = base_name
            else:
                metadata["title"] = doc_type
        
        metadata["document_type"] = doc_type
        
        # Extract primary pregnancy stages
        primary_stages = []
        for stage, keywords in self.pregnancy_stages.items():
            if any(keyword in text_lower for keyword in keywords):
                primary_stages.append(stage)
        
        metadata["primary_stages"] = primary_stages
        
        return metadata
    
    def extract_pregnancy_stages(self, text: str) -> Dict[str, List]:
        """
        Extract all pregnancy stages mentioned in the text.
        
        Args:
            text: Text to analyze
            
        Returns:
            Dict with pregnancy stage information
        """
        stages = {
            "weeks": [],
            "months": [],
            "trimesters": [],
            "named_stages": []
        }
        
        # Extract weeks
        for match in self.stage_patterns['week'].finditer(text):
            week_str = match.group(1) or match.group(2)
            if week_str:
                if '-' in week_str:
                    try:
                        start, end = map(int, week_str.split('-'))
                        stages["weeks"].extend(list(range(start, end + 1)))
                    except ValueError:
                        pass
                else:
                    try:
                        stages["weeks"].append(int(week_str))
                    except ValueError:
                        pass
        
        # Extract months
        for match in self.stage_patterns['month'].finditer(text):
            month_str = match.group(1) or match.group(2)
            if month_str:
                if '-' in month_str:
                    try:
                        start, end = map(int, month_str.split('-'))
                        stages["months"].extend(list(range(start, end + 1)))
                    except ValueError:
                        pass
                else:
                    try:
                        stages["months"].append(int(month_str))
                    except ValueError:
                        pass
        
        # Extract trimesters
        for match in self.stage_patterns['trimester'].finditer(text):
            trimester = match.group(1).lower()
            stages["trimesters"].append(trimester)
        
        # Extract named stages
        for match in self.stage_patterns['stage'].finditer(text):
            named_stage = match.group(1).lower()
            stages["named_stages"].append(named_stage)
        
        # Remove duplicates
        for key in stages:
            stages[key] = list(set(stages[key]))
            
        # Map to standard stage categorization
        standard_stages = set()
        
        # Map from weeks to standard stages
        for week in stages["weeks"]:
            if 1 <= week <= 12:
                standard_stages.add("first_trimester")
            elif 13 <= week <= 26:
                standard_stages.add("second_trimester")
            elif 27 <= week <= 42:
                standard_stages.add("third_trimester")
        
        # Map from months to standard stages
        for month in stages["months"]:
            if 1 <= month <= 3:
                standard_stages.add("first_trimester")
            elif 4 <= month <= 6:
                standard_stages.add("second_trimester")
            elif 7 <= month <= 10:
                standard_stages.add("third_trimester")
        
        # Map from named trimesters
        for trimester in stages["trimesters"]:
            if trimester == "first":
                standard_stages.add("first_trimester")
            elif trimester == "second":
                standard_stages.add("second_trimester")
            elif trimester == "third":
                standard_stages.add("third_trimester")
        
        # Map from named stages
        for stage in stages["named_stages"]:
            if stage in ["preconception", "pre-conception"]:
                standard_stages.add("preconception")
            elif stage in ["prenatal", "antenatal"]:
                # These are general terms - don't add a specific stage
                pass
            elif stage == "perinatal":
                standard_stages.add("labor_delivery")
            elif stage == "intrapartum":
                standard_stages.add("labor_delivery")
            elif stage in ["postpartum", "postnatal"]:
                standard_stages.add("postpartum")
        
        stages["standard_stages"] = list(standard_stages)
        
        return stages
    
    def extract_medical_entities(self, text: str) -> Dict[str, List[str]]:
        """
        Extract medical entities from text using pattern matching and NLP.
        
        Args:
            text: Text to analyze
            
        Returns:
            Dict with medical entities by category
        """
        text_lower = text.lower()
        
        entities = {
            "conditions": [],
            "medications": [],
            "procedures": [],
            "measurements": []
        }
        
        # Extract entities using pattern matching
        for entity_type, entity_list in self.medical_entities.items():
            for entity in entity_list:
                if entity in text_lower:
                    entities[entity_type].append(entity)
        
        # Extract measurements using regex
        for match in self.clinical_patterns['measurement'].finditer(text):
            measurement = match.group(0)
            if measurement not in entities["measurements"]:
                entities["measurements"].append(measurement)
        
        # Extract dosages using regex
        for match in self.clinical_patterns['dosage'].finditer(text):
            dosage = match.group(0)
            if dosage not in entities["medications"]:
                entities["medications"].append(dosage)
        
        # Use SpaCy for enhanced entity extraction if available
        if self.use_spacy and self.nlp is not None:
            try:
                # Process with SpaCy - limit text length to avoid memory issues
                # Truncate to ~10,000 chars for efficiency
                truncated_text = text[:10000] if len(text) > 10000 else text
                doc = self.nlp(truncated_text)
                
                # Extract medical entities from SpaCy
                for ent in doc.ents:
                    ent_text = ent.text.lower()
                    
                    # Map SpaCy entities to our categories
                    if ent.label_ in ["CONDITION", "DISEASE", "PROBLEM"]:
                        if ent_text not in entities["conditions"]:
                            entities["conditions"].append(ent_text)
                    elif ent.label_ in ["TREATMENT", "PROCEDURE"]:
                        if ent_text not in entities["procedures"]:
                            entities["procedures"].append(ent_text)
                    elif ent.label_ in ["MEDICATION", "DRUG"]:
                        if ent_text not in entities["medications"]:
                            entities["medications"].append(ent_text)
                    elif ent.label_ in ["QUANTITY"] and any(unit in ent_text for unit in ["mg", "kg", "cm", "mmHg", "bpm"]):
                        if ent_text not in entities["measurements"]:
                            entities["measurements"].append(ent_text)
            except Exception as e:
                logger.warning(f"Error during SpaCy processing: {e}")
        
        return entities
    
    def identify_topics_and_concerns(self, text: str) -> Dict[str, List[str]]:
        """
        Identify maternal health topics and patient concerns in text.
        
        Args:
            text: Text to analyze
            
        Returns:
            Dict with identified topics and concerns
        """
        text_lower = text.lower()
        results = {
            "topics": [],
            "concerns": [],
            "content_types": []
        }
        
        # Identify topics
        for topic, keywords in self.maternal_topics.items():
            if any(keyword in text_lower for keyword in keywords):
                results["topics"].append(topic)
        
        # Identify patient concerns
        for concern, keywords in self.patient_concerns.items():
            if any(keyword in text_lower for keyword in keywords):
                results["concerns"].append(concern)
        
        # Identify content types
        content_indicators = {
            "warning": ["warning", "caution", "danger", "alert", "emergency", "attention"],
            "recommendation": ["recommend", "should", "advised", "advise", "important", "must"],
            "information": ["information", "facts", "overview", "background", "introduction"],
            "instruction": ["step", "steps", "procedure", "instructions", "follow", "method"],
            "faq": ["question", "answer", "frequently", "asked", "faq", "q&a", "q:", "a:"]
        }
        
        for content_type, indicators in content_indicators.items():
            if any(indicator in text_lower for indicator in indicators):
                results["content_types"].append(content_type)
        
        # Check for lists and tables using regex
        if self.preservation_patterns['list'].search(text):
            results["content_types"].append("list")
        
        if self.preservation_patterns['table'].search(text):
            results["content_types"].append("table")
        
        return results
    
    def split_text_into_sections(self, text: str) -> List[Tuple[str, str]]:
        """
        Split text into semantically meaningful sections with headers.
        
        Args:
            text: Full document text
            
        Returns:
            List of (header, section_text) tuples
        """
        # If text is small enough, don't bother splitting
        if len(text) < 5000:
            return [("", text)]
        
        # Find all semantic boundaries using header patterns
        header_positions = []
        
        # Collect positions of all types of headers
        for pattern_name, pattern in self.structure_patterns.items():
            if pattern_name in ['formal_header', 'nl_header', 'special_marker']:
                for match in pattern.finditer(text):
                    header_positions.append((match.start(), match.group(0)))
        
        # Sort header positions by their occurrence in text
        header_positions.sort(key=lambda x: x[0])
        
        # Filter out headers that are too close together (likely not real section headers)
        if header_positions:
            filtered_positions = [(0, "")]  # Start with the beginning of text
            for pos, header in header_positions:
                # Only keep headers that are at least 500 chars from the previous header
                if pos - filtered_positions[-1][0] >= 500:
                    filtered_positions.append((pos, header))
            
            # If we only have the start position, there are no useful headers
            if len(filtered_positions) == 1:
                return [("", text)]
            
            # Create sections based on header positions
            sections = []
            for i in range(len(filtered_positions) - 1):
                start_pos, header = filtered_positions[i]
                end_pos = filtered_positions[i+1][0]
                
                # Skip the first "header" which is just the start position
                if i == 0 and start_pos == 0 and header == "":
                    continue
                
                section_text = text[start_pos:end_pos].strip()
                if section_text:
                    sections.append((header.strip(), section_text))
            
            # Don't forget the last section
            last_pos, last_header = filtered_positions[-1]
            last_section = text[last_pos:].strip()
            if last_section:
                sections.append((last_header.strip(), last_section))
            
            return sections if sections else [("", text)]
        else:
            # If no headers detected, return the whole text
            return [("", text)]
    
    def find_preservation_blocks(self, text: str) -> List[Tuple[int, int, str, str]]:
        """
        Find blocks of text that should be preserved during chunking.
        
        Args:
            text: Text to analyze
            
        Returns:
            List of (start_pos, end_pos, block_text, block_type) tuples
        """
        preserved_blocks = []
        
        # Check each preservation pattern
        for block_type, pattern in self.preservation_patterns.items():
            for match in pattern.finditer(text):
                preserved_blocks.append((match.start(), match.end(), match.group(0), block_type))
        
        # Sort by start position
        preserved_blocks.sort(key=lambda x: x[0])
        
        return preserved_blocks
    
    def create_semantic_chunks(self, section_text: str, section_header: str, doc_metadata: Dict[str, Any]) -> List[Document]:
        """
        Create semantically meaningful chunks from a section of text.
        
        Args:
            section_text: Text of the section to chunk
            section_header: Header of the section
            doc_metadata: Document-level metadata
            
        Returns:
            List of Document objects with chunks and metadata
        """
        # If section is small enough, keep it as one chunk
        if len(section_text) <= self.chunk_size:
            return [self._create_chunk_document(section_text, section_header, doc_metadata)]
        
        # Find blocks that should be preserved
        preserved_blocks = self.find_preservation_blocks(section_text)
        
        chunks = []
        
        # If we have SpaCy available, use sentence boundaries for better chunking
        if self.use_spacy and self.nlp is not None:
            # Handle preserved blocks first
            if preserved_blocks:
                # Create a map of positions to skip (preserved blocks)
                skip_ranges = [(start, end) for start, end, _, _ in preserved_blocks]
                
                # Process text outside preserved blocks with sentence boundaries
                current_pos = 0
                current_chunk = []
                current_length = 0
                
                try:
                    # Process text in chunks to avoid memory issues with very large texts
                    segment_length = 50000  # Process 50k chars at a time
                    for i in range(0, len(section_text), segment_length):
                        segment = section_text[i:i+segment_length]
                        
                        # Adjust positions to account for the segment offset
                        segment_offset = i
                        
                        # Process with SpaCy for sentence boundaries
                        doc = self.nlp(segment)
                        sentences = list(doc.sents)
                        
                        for sentence in sentences:
                            # Adjust sentence positions to absolute document position
                            abs_start = sentence.start_char + segment_offset
                            abs_end = sentence.end_char + segment_offset
                            
                            # Check if this sentence overlaps with any preserved block
                            in_preserved_block = False
                            for start, end in skip_ranges:
                                if (start <= abs_start < end) or (start < abs_end <= end) or (abs_start <= start and abs_end >= end):
                                    in_preserved_block = True
                                    break
                            
                            if in_preserved_block:
                                continue
                                
                            sentence_text = sentence.text.strip()
                            
                            # If adding this sentence exceeds chunk size, create a new chunk
                            if current_length + len(sentence_text) > self.chunk_size and current_chunk:
                                chunk_text = ' '.join(current_chunk)
                                chunks.append(self._create_chunk_document(chunk_text, section_header, doc_metadata))
                                current_chunk = []
                                current_length = 0
                            
                            # Add sentence to current chunk
                            current_chunk.append(sentence_text)
                            current_length += len(sentence_text) + 1  # +1 for space
                        
                except Exception as e:
                    logger.warning(f"Error during SpaCy processing: {e}")
                    # Fall back to simpler chunking method
                    return self._simple_chunk_section(section_text, section_header, doc_metadata, preserved_blocks)
                
                # Add final chunk
                if current_chunk:
                    chunk_text = ' '.join(current_chunk)
                    chunks.append(self._create_chunk_document(chunk_text, section_header, doc_metadata))
                
                # Now add preserved blocks as separate chunks
                for start, end, block_text, block_type in preserved_blocks:
                    block_metadata = doc_metadata.copy()
                    block_metadata["content_type"] = block_type
                    chunks.append(self._create_chunk_document(block_text, section_header, block_metadata))
            else:
                # No preserved blocks, process entire text with sentence boundaries
                try:
                    # Process text in chunks to avoid memory issues
                    current_chunk = []
                    current_length = 0
                    
                    # Process text in segments
                    segment_length = 50000  # Process 50k chars at a time
                    for i in range(0, len(section_text), segment_length):
                        segment = section_text[i:i+segment_length]
                        
                        # Process with SpaCy for sentence boundaries
                        doc = self.nlp(segment)
                        sentences = list(doc.sents)
                        
                        for sentence in sentences:
                            sentence_text = sentence.text.strip()
                            
                            # If adding this sentence exceeds chunk size, create a new chunk
                            if current_length + len(sentence_text) > self.chunk_size and current_chunk:
                                chunk_text = ' '.join(current_chunk)
                                chunks.append(self._create_chunk_document(chunk_text, section_header, doc_metadata))
                                current_chunk = []
                                current_length = 0
                            
                            # Add sentence to current chunk
                            current_chunk.append(sentence_text)
                            current_length += len(sentence_text) + 1  # +1 for space
                    
                    # Add final chunk
                    if current_chunk:
                        chunk_text = ' '.join(current_chunk)
                        chunks.append(self._create_chunk_document(chunk_text, section_header, doc_metadata))
                        
                except Exception as e:
                    logger.warning(f"Error during SpaCy processing: {e}")
                    # Fall back to simpler chunking method
                    return self._simple_chunk_section(section_text, section_header, doc_metadata, preserved_blocks)
        else:
            # SpaCy not available, use simpler chunking
            chunks = self._simple_chunk_section(section_text, section_header, doc_metadata, preserved_blocks)
        
        return chunks
    
    def _simple_chunk_section(self, text: str, header: str, metadata: Dict[str, Any], 
                            preserved_blocks: List[Tuple[int, int, str, str]]) -> List[Document]:
        """
        Chunk text using paragraph boundaries when SpaCy is not available.
        
        Args:
            text: Text to chunk
            header: Section header
            metadata: Document metadata
            preserved_blocks: List of preserved blocks
            
        Returns:
            List of Document objects
        """
        chunks = []
        
        # Handle preserved blocks first
        if preserved_blocks:
            # Sort preserved blocks by position
            preserved_blocks.sort(key=lambda x: x[0])
            
            # Keep track of the current position
            current_pos = 0
            
            # Process text segments between preserved blocks
            for start, end, block_text, block_type in preserved_blocks:
                # Handle text before this preserved block
                if start > current_pos:
                    preceding_text = text[current_pos:start].strip()
                    if preceding_text:
                        # Chunk this text by paragraphs
                        preceding_chunks = self._chunk_by_paragraphs(preceding_text, header, metadata)
                        chunks.extend(preceding_chunks)
                
                # Add the preserved block as its own chunk
                block_metadata = metadata.copy()
                block_metadata["content_type"] = block_type
                chunks.append(self._create_chunk_document(block_text, header, block_metadata))
                
                # Update current position
                current_pos = end
            
            # Handle text after the last preserved block
            if current_pos < len(text):
                remaining_text = text[current_pos:].strip()
                if remaining_text:
                    # Chunk this text by paragraphs
                    remaining_chunks = self._chunk_by_paragraphs(remaining_text, header, metadata)
                    chunks.extend(remaining_chunks)
        else:
            # No preserved blocks, chunk entire text by paragraphs
            chunks = self._chunk_by_paragraphs(text, header, metadata)
        
        return chunks
    
    def _chunk_by_paragraphs(self, text: str, header: str, metadata: Dict[str, Any]) -> List[Document]:
        """
        Split text into chunks based on paragraph boundaries.
        
        Args:
            text: Text to chunk
            header: Section header
            metadata: Document metadata
            
        Returns:
            List of Document objects
        """
        chunks = []
        
        # Split on paragraph boundaries (double newlines)
        paragraphs = re.split(r'\n\s*\n', text)
        
        current_chunk = []
        current_length = 0
        
        for paragraph in paragraphs:
            paragraph = paragraph.strip()
            if not paragraph:
                continue
                
            # If adding this paragraph would exceed chunk size and we already have content,
            # create a new chunk
            if current_length + len(paragraph) > self.chunk_size and current_chunk:
                chunk_text = '\n\n'.join(current_chunk)
                chunks.append(self._create_chunk_document(chunk_text, header, metadata))
                current_chunk = []
                current_length = 0
            
            # If a single paragraph is larger than chunk size, we need to split it
            if len(paragraph) > self.chunk_size:
                # If we have accumulated content, create a chunk first
                if current_chunk:
                    chunk_text = '\n\n'.join(current_chunk)
                    chunks.append(self._create_chunk_document(chunk_text, header, metadata))
                    current_chunk = []
                    current_length = 0
                
                # Split the large paragraph by sentences (using basic regex)
                sentences = re.split(r'(?<=[.!?])\s+', paragraph)
                
                sentence_chunk = []
                sentence_length = 0
                
                for sentence in sentences:
                    # If this sentence alone exceeds chunk size, we have to include it anyway
                    # Users will just have a larger-than-ideal chunk in this case
                    if sentence_length + len(sentence) > self.chunk_size and sentence_chunk:
                        sentence_text = ' '.join(sentence_chunk)
                        chunks.append(self._create_chunk_document(sentence_text, header, metadata))
                        sentence_chunk = []
                        sentence_length = 0
                    
                    sentence_chunk.append(sentence)
                    sentence_length += len(sentence) + 1  # +1 for space
                
                # Add final sentence chunk if any
                if sentence_chunk:
                    sentence_text = ' '.join(sentence_chunk)
                    chunks.append(self._create_chunk_document(sentence_text, header, metadata))
            else:
                # Normal sized paragraph
                current_chunk.append(paragraph)
                current_length += len(paragraph) + 2  # +2 for newlines
        
        # Add final chunk
        if current_chunk:
            chunk_text = '\n\n'.join(current_chunk)
            chunks.append(self._create_chunk_document(chunk_text, header, metadata))
        
        return chunks
    
    def _create_chunk_document(self, text: str, header: str, base_metadata: Dict[str, Any]) -> Document:
        """
        Create a document object with metadata from chunk text.
        
        Args:
            text: Chunk text
            header: Section header
            base_metadata: Base document metadata
            
        Returns:
            Document object with metadata
        """
        # Create a copy of base metadata
        metadata = base_metadata.copy()
        
        # Add section header
        if header:
            metadata["section_header"] = header
        
        # Extract pregnancy stages
        stages = self.extract_pregnancy_stages(text)
        metadata["pregnancy_stages"] = stages
        
        # Extract medical entities
        entities = self.extract_medical_entities(text)
        metadata["medical_entities"] = entities
        
        # Identify topics and concerns
        topics_and_concerns = self.identify_topics_and_concerns(text)
        metadata["topics"] = topics_and_concerns["topics"]
        metadata["patient_concerns"] = topics_and_concerns["concerns"]
        
        # Set content type if not already set
        if "content_type" not in metadata:
            content_types = topics_and_concerns["content_types"]
            if content_types:
                metadata["content_type"] = content_types[0]  # Primary content type
                metadata["all_content_types"] = content_types
            else:
                metadata["content_type"] = "general_text"
        
        # Create unique ID for the chunk
        metadata["chunk_id"] = str(uuid.uuid4())
        
        return Document(page_content=text, metadata=metadata)
    
    def create_synthetic_chunks(self, documents: List[Document]) -> List[Document]:
        """
        Create synthetic chunks for improved retrieval.
        
        Args:
            documents: Original document chunks
            
        Returns:
            List of synthetic Document objects
        """
        synthetic_chunks = []
        logger.info("Creating synthetic chunks for improved retrieval...")
        
        # Group documents by stage
        stage_groups = {}
        
        for doc in documents:
            stages = doc.metadata.get('pregnancy_stages', {}).get('standard_stages', [])
            if not stages:
                continue
                
            for stage in stages:
                if stage not in stage_groups:
                    stage_groups[stage] = []
                stage_groups[stage].append(doc)
        
        # Create stage+topic chunks
        for stage, stage_docs in stage_groups.items():
            # Skip if too few documents
            if len(stage_docs) < 3:
                continue
                
            # Group by topics within stage
            topic_groups = {}
            
            for doc in stage_docs:
                for topic in doc.metadata.get('topics', []):
                    if topic not in topic_groups:
                        topic_groups[topic] = []
                    topic_groups[topic].append(doc)
            
            # Create synthetic chunks for each topic with sufficient content
            for topic, topic_docs in topic_groups.items():
                if len(topic_docs) < 2:
                    continue
                    
                # Select best content (at most 5 chunks or 5000 chars)
                selected_docs = []
                total_length = 0
                
                # Prioritize docs with specific content types
                for content_type in ["recommendation", "information", "table", "list"]:
                    for doc in topic_docs:
                        if doc.metadata.get('content_type') == content_type:
                            if len(selected_docs) < 5 and total_length < 5000:
                                selected_docs.append(doc)
                                total_length += len(doc.page_content)
                
                # Add other docs if needed
                for doc in topic_docs:
                    if doc not in selected_docs:
                        if len(selected_docs) < 5 and total_length < 5000:
                            selected_docs.append(doc)
                            total_length += len(doc.page_content)
                
                if not selected_docs:
                    continue
                
                # Create synthetic content
                synthetic_content = f"# {stage.replace('_', ' ').title()} - {topic.replace('_', ' ').title()}\n\n"
                
                for i, doc in enumerate(selected_docs):
                    # Add section header if available
                    if "section_header" in doc.metadata:
                        synthetic_content += f"## {doc.metadata['section_header']}\n\n"
                    else:
                        synthetic_content += f"## Information {i+1}\n\n"
                    
                    synthetic_content += doc.page_content + "\n\n"
                
                # Create synthetic document
                synthetic_doc = Document(
                    page_content=synthetic_content,
                    metadata={
                        "document_type": "Synthetic Document",
                        "title": f"{stage.replace('_', ' ').title()} - {topic.replace('_', ' ').title()}",
                        "is_synthetic": True,
                        "source_stage": stage,
                        "source_topic": topic,
                        "content_type": "synthetic",
                        "pregnancy_stages": {
                            "standard_stages": [stage]
                        },
                        "topics": [topic],
                        "source_count": len(selected_docs),
                        "chunk_id": f"synthetic_{stage}_{topic}_{uuid.uuid4().hex[:8]}"
                    }
                )
                
                synthetic_chunks.append(synthetic_doc)
        
        # Create FAQ chunks for common questions
        synthetic_chunks.extend(self._create_faq_chunks(documents))
        
        logger.info(f"Created {len(synthetic_chunks)} synthetic chunks")
        return synthetic_chunks
    
    def _create_faq_chunks(self, documents: List[Document]) -> List[Document]:
        """
        Create FAQ-oriented chunks for common patient questions.
        
        Args:
            documents: Original documents
            
        Returns:
            List of FAQ Document objects
        """
        faq_chunks = []
        
        # Common questions by stage and concern
        faq_templates = {
            "first_trimester": [
                ("What symptoms are normal in the first trimester?", ["symptoms", "normal_variations", "maternal_changes"]),
                ("What foods should I avoid in early pregnancy?", ["nutrition", "warning_signs"]),
                ("Is spotting normal in early pregnancy?", ["bleeding", "normal_variations"])
            ],
            "second_trimester": [
                ("When will I feel the baby move?", ["movement", "fetal_development"]),
                ("What tests are done in the second trimester?", ["medical_tests", "prenatal_care"]),
                ("Is it normal to have pain in my abdomen as my belly grows?", ["pain", "normal_variations"])
            ],
            "third_trimester": [
                ("What are signs of labor?", ["delivery", "warning_signs"]),
                ("How can I relieve back pain in late pregnancy?", ["pain", "maternal_changes"]),
                ("How do I know if my water broke?", ["delivery"])
            ],
            "postpartum": [
                ("How much bleeding is normal after birth?", ["bleeding", "normal_variations", "postpartum"]),
                ("How can I tell if I have postpartum depression?", ["mental_health", "postpartum"]),
                ("When will my milk come in?", ["postpartum", "breastfeeding"])
            ]
        }
        
        # Create FAQs for each stage
        for stage, questions in faq_templates.items():
            for question, topics in questions:
                # Find relevant documents
                relevant_docs = []
                
                for doc in documents:
                    # Check if the document matches stage and topics
                    doc_stages = doc.metadata.get('pregnancy_stages', {}).get('standard_stages', [])
                    doc_topics = doc.metadata.get('topics', [])
                    doc_concerns = doc.metadata.get('patient_concerns', [])
                    
                    # Match if document has the right stage and at least one matching topic
                    if stage in doc_stages and any(topic in doc_topics for topic in topics):
                        relevant_docs.append(doc)
                
                # Skip if insufficient relevant documents
                if len(relevant_docs) < 2:
                    continue
                
                # Select best content (maximum 4 docs or 4000 chars)
                selected_docs = []
                total_length = 0
                
                # First add recommendations and key information
                for doc in sorted(relevant_docs, 
                                 key=lambda d: 1 if d.metadata.get('content_type') == 'recommendation' else 2):
                    if len(selected_docs) < 4 and total_length < 4000:
                        selected_docs.append(doc)
                        total_length += len(doc.page_content)
                
                # Create FAQ content
                faq_content = f"# FAQ: {question}\n\n"
                faq_content += "## Answer\n\n"
                
                # Add content from selected documents
                for doc in selected_docs:
                    faq_content += doc.page_content + "\n\n"
                
                # Create FAQ document
                faq_doc = Document(
                    page_content=faq_content,
                    metadata={
                        "document_type": "FAQ",
                        "title": question,
                        "is_faq": True,
                        "question": question,
                        "source_stage": stage,
                        "source_topics": topics,
                        "content_type": "faq",
                        "pregnancy_stages": {
                            "standard_stages": [stage]
                        },
                        "topics": topics,
                        "chunk_id": f"faq_{stage}_{uuid.uuid4().hex[:8]}"
                    }
                )
                
                faq_chunks.append(faq_doc)
        
        return faq_chunks
    
    def process_document(self, file_path: str) -> List[Document]:
        """
        Process a single document into semantically meaningful chunks.
        
        Args:
            file_path: Path to document
            
        Returns:
            List of Document objects
        """
        logger.info(f"Processing {file_path}...")
        
        # Load document text
        try:
            if file_path.lower().endswith('.pdf'):
                loader = PyPDFLoader(file_path)
                pages = loader.load()
                text = "\n\n".join([page.page_content for page in pages])
            elif file_path.lower().endswith('.html'):
                with open(file_path, "r", encoding="utf-8", errors="replace") as f:
                    html = f.read()
                soup = BeautifulSoup(html, "html.parser")
                # get_text keeps readability; separator inserts newlines for headings/paras
                text = soup.get_text(separator="\n")
            else:
                with open(file_path, 'r', encoding='utf-8', errors='replace') as f:
                    text = f.read()
        except Exception as e:
            logger.error(f"Error loading {file_path}: {e}")
            return []
        
        # Extract document-level metadata
        filename = os.path.basename(file_path)
        doc_metadata = self._extract_document_metadata(text[:10000], filename)
        
        # Split into sections
        logger.info(f"Splitting {filename} into sections...")
        sections = self.split_text_into_sections(text)
        logger.info(f"Found {len(sections)} sections")
        
        # Process each section into chunks
        all_chunks = []
        
        for header, section_text in tqdm(sections, desc="Processing sections"):
            chunks = self.create_semantic_chunks(section_text, header, doc_metadata)
            all_chunks.extend(chunks)
        
        logger.info(f"Created {len(all_chunks)} chunks from {filename}")
        return all_chunks
    
    def process_directory(self, directory_path: str) -> List[Document]:
        """
        Process all documents in a directory.
        
        Args:
            directory_path: Path to directory
            
        Returns:
            List of all Document objects
        """
        all_documents = []
        
        # Get all files in directory
        file_paths = []
        for filename in os.listdir(directory_path):
            file_path = os.path.join(directory_path, filename)
            if os.path.isfile(file_path) and not filename.startswith('.'):
                file_paths.append(file_path)
        
        # Process each file
        logger.info(f"Processing {len(file_paths)} files in {directory_path}")
        
        for file_path in tqdm(file_paths, desc="Processing files"):
            try:
                chunks = self.process_document(file_path)
                all_documents.extend(chunks)
            except Exception as e:
                logger.error(f"Error processing {file_path}: {e}")
        
        return all_documents
    
    def create_vector_store(self, documents: List[Document], include_synthetic: bool = True) -> FAISS:
        """
        Create vector store from processed documents.
        
        Args:
            documents: List of Document objects
            include_synthetic: Whether to include synthetic chunks
            
        Returns:
            FAISS vector store
        """
        if not documents:
            raise ValueError("No documents provided to create vector store")
        
        # Create synthetic chunks if requested
        if include_synthetic:
            synthetic_chunks = self.create_synthetic_chunks(documents)
            all_docs = documents + synthetic_chunks
        else:
            all_docs = documents
        
        logger.info(f"Creating vector store with {len(all_docs)} documents...")
        
        # Initialize FAISS index from documents
        self.db = FAISS.from_documents(
            documents=all_docs,
            embedding=self.embeddings
        )
        
        # Save to disk
        logger.info(f"Saving vector store to {self.db_directory}")
        self.db.save_local(self.db_directory)
        
        return self.db
    
    def create_stage_optimized_retrievers(self) -> Dict[str, Any]:
        """Create stage-optimized retrievers for each pregnancy stage."""
        if not self.db:
            raise ValueError("No vector store available. Please create or load a vector store first.")
        
        # Define standard pregnancy stages
        stages = [
            "preconception",
            "first_trimester",
            "second_trimester", 
            "third_trimester",
            "labor_delivery",
            "postpartum"
        ]
        
        retrievers = {}
        
        # Create a retriever for each stage with metadata filtering
        for stage in stages:
            # Define filter function that works with metadata dictionaries
            def stage_filter(metadata):
                # Get pregnancy stages from metadata
                pregnancy_stages = metadata.get("pregnancy_stages", {})
                if isinstance(pregnancy_stages, dict):
                    standard_stages = pregnancy_stages.get("standard_stages", [])
                else:
                    standard_stages = []
                    
                # Check if content is stage-agnostic
                is_general_content = (
                    not standard_stages and 
                    metadata.get("content_type") in ["recommendation", "information"]
                )
                    
                # Include if stage matches or it's general content
                return stage in standard_stages or is_general_content
            
            # Create retriever with this filter
            retrievers[stage] = self.db.as_retriever(
                search_kwargs={"k": 7, "filter": stage_filter}
            )
        
        # Create a general retriever for queries without clear stage
        retrievers["general"] = self.db.as_retriever(
            search_kwargs={"k": 5}
        )
    
        return retrievers
    
    def detect_query_context(self, query: str) -> Dict[str, Any]:
        """
        Detect pregnancy stage and concerns from user query.
        
        Args:
            query: User query text
            
        Returns:
            Dict with detected context
        """
        context = {
            "stage": None,
            "concerns": [],
            "detected_topics": []
        }
        
        # Detect pregnancy stage
        # First look for explicit stage mentions
        query_lower = query.lower()
        
        for stage, keywords in self.pregnancy_stages.items():
            if any(keyword in query_lower for keyword in keywords):
                context["stage"] = stage
                break
        
        # If no explicit stage, check for week/month/trimester mentions
        if not context["stage"]:
            stages = self.extract_pregnancy_stages(query)
            if stages["standard_stages"]:
                context["stage"] = stages["standard_stages"][0]
        
        # Detect concerns
        for concern, keywords in self.patient_concerns.items():
            if any(keyword in query_lower for keyword in keywords):
                context["concerns"].append(concern)
        
        # Detect topics
        for topic, keywords in self.maternal_topics.items():
            if any(keyword in query_lower for keyword in keywords):
                context["detected_topics"].append(topic)
        
        return context
    
    def retrieve_for_query(self, query: str) -> List[Document]:
        """Smart retrieval based on query context."""
        if not self.db:
            raise ValueError("No vector store available. Please create or load a vector store first.")
        
        # Detect query context
        context = self.detect_query_context(query)
        logger.info(f"Query context: {context}")
        
        # Create stage-specific retrievers if not already created
        if not hasattr(self, 'stage_retrievers'):
            self.stage_retrievers = self.create_stage_optimized_retrievers()
        
        # Select appropriate retriever based on context
        if context["stage"]:
            retriever = self.stage_retrievers.get(context["stage"])
            if not retriever:
                # Fall back to general retriever
                retriever = self.stage_retrievers["general"]
        else:
            # Use general retriever for stage-agnostic queries
            retriever = self.stage_retrievers["general"]
        
        # Get relevant documents
        docs = retriever.get_relevant_documents(query)
        
        # If we have specific concerns, try to ensure we have documents addressing them
        if context["concerns"] and len(docs) >= 3:
            # Check if concerns are addressed in retrieved docs
            concerns_addressed = False
            for doc in docs[:3]:  # Check top 3 docs
                doc_concerns = doc.metadata.get("patient_concerns", [])
                if any(concern in doc_concerns for concern in context["concerns"]):
                    concerns_addressed = True
                    break
        
        # If concerns not addressed in top results, add specific concern filters
            if not concerns_addressed:
                # Updated filter function to work with metadata dictionaries
                def concern_filter(metadata):
                    # Get patient concerns from metadata
                    patient_concerns = metadata.get("patient_concerns", [])
                    # Check if any of our concerns are in the document's concerns
                    return any(concern in patient_concerns for concern in context["concerns"])
                
                concern_retriever = self.db.as_retriever(
                    search_kwargs={"k": 2, "filter": concern_filter}
                )
                
                # Get concern-specific docs and add to results
                concern_docs = concern_retriever.get_relevant_documents(query)
                
                # Add unique concern docs (avoid duplicates)
                existing_ids = {doc.metadata.get("chunk_id", "") for doc in docs}
                for doc in concern_docs:
                    if doc.metadata.get("chunk_id", "") not in existing_ids:
                        docs.append(doc)
        
        return docs
    
    def create_qa_chain(self, model_name: str = "gpt-4", temperature: float = 0.0) -> RetrievalQA:
        """
        Create a question answering chain.
        
        Args:
            model_name: LLM model name
            temperature: LLM temperature
            
        Returns:
            RetrievalQA chain
        """
        if not self.db:
            raise ValueError("No vector store available. Please create or load a vector store first.")
        
        # Initialize LLM
        llm = ChatOpenAI(
            model=model_name,
            temperature=temperature
        )
        
        # Create custom prompt for maternal health
        qa_prompt = PromptTemplate(
            template="""You are a maternal health assistant providing evidence-based information to pregnant women and new mothers.
            Use the following retrieved information to answer the question.
            
            If the information doesn't contain the answer, just say "I don't have enough information to answer that question accurately" - don't make up information.
            If providing medical advice, clearly state that this is general information and the person should consult their healthcare provider.
            Use a reassuring, supportive tone and clear, non-technical language.
            
            Retrieved information:
            {context}
            
            Question: {question}
            
            Answer:""",
            input_variables=["context", "question"]
        )
        
        # Create question answering chain
        qa_chain = RetrievalQA.from_chain_type(
            llm=llm,
            chain_type="stuff",
            retriever=self.db.as_retriever(search_kwargs={"k": 5}),
            chain_type_kwargs={"prompt": qa_prompt},
            return_source_documents=True
        )
        
        return qa_chain
    
    def answer_question(self, question: str, model_name: str = "gpt-4") -> Dict[str, Any]:
        """
        Answer a maternal health question using the optimized retrieval pipeline.
        
        Args:
            question: User question
            model_name: LLM model name
            
        Returns:
            Dict with answer and source documents
        """
        # Retrieve relevant documents with context-aware retrieval
        docs = self.retrieve_for_query(question)
        
        # Initialize LLM
        llm = ChatOpenAI(
            model=model_name,
            temperature=0.1  # Low temperature for factual responses
        )
        
        # Create maternal health-specific prompt
        # prompt = PromptTemplate(
        #     template="""You are a supportive maternal health assistant providing evidence-based information to pregnant women and new mothers.
            
        #     Use the following retrieved information to answer the question.
            
        #     Guidelines for your response:
        #     1. Be accurate - only use information from the provided context
        #     2. If the information doesn't fully answer the question, acknowledge the limitations
        #     3. Be reassuring but honest - don't minimize legitimate concerns
        #     4. Use clear, non-technical language
        #     5. Remind users to consult healthcare providers for personal medical advice
        #     6. If discussing warning signs, clearly distinguish between normal variations and concerning symptoms
            
        #     Retrieved information:
        #     {context}
            
        #     Question: {question}
            
        #     Answer:""",
        #     input_variables=["context", "question"]
        # )
        prompt = PromptTemplate(
            template="""You are a supportive maternal health assistant providing evidence-based information to pregnant women and new mothers.
Use the following retrieved information to answer the question.
Response Guidelines:
Answer ONLY using the provided context. Do not speculate or generate information not found in the retrieved data.


If the information doesn't fully answer the question, state the limitations. Example: “I don’t have enough information to answer that accurately.”


Be reassuring but honest. Do not minimize legitimate concerns. If the topic is serious, clearly explain when to seek medical help.


Use clear, non-technical language. Avoid jargon and explain complex terms simply.


Always remind users: “This is general information; please consult your healthcare provider for personal medical advice.”


Warning signs: When discussing symptoms or warning signs, clearly distinguish between normal variations and symptoms that require medical attention. List these separately if appropriate.


If you detect any signs of a possible emergency (such as heavy bleeding, severe pain, loss of consciousness, seizures, absent fetal movement, severe swelling, difficulty breathing, fever with rash, premature rupture of membranes, or vision changes):


Do NOT give any advice.


Respond ONLY with:


"IMPORTANT: This appears to be an emergency situation that requires immediate medical attention.
This chatbot cannot provide emergency medical advice. Please:
1. Contact your healthcare provider immediately OR
2. Go to the nearest emergency department OR
3. Call your local emergency number for immediate assistance.

Do not wait or delay seeking professional care."

Do not provide:


Any specific drug brand or prescription recommendations.


Any information related to fetal gender detection or selection.


Any answers that are not about pregnancy, postpartum, or maternal/newborn wellness (if off-topic, say: “I’m sorry, I can only help with pregnancy and maternal-health questions.”)


Any instructions or advice for suicidal/self-harm situations. Instead, respond only with the relevant crisis hotline (India: 9152987821, US: 988, UK: 0800 689 5652).



Retrieved information:
 {context}
Question:
 {question}
Answer:

""",
            input_variables=["context", "question"]
        )
        # Create QA chain
        qa_chain = RetrievalQA.from_chain_type(
            llm=llm,
            chain_type="stuff",
            retriever=self.db.as_retriever(),  # Default retriever not used since we pass docs directly
            chain_type_kwargs={"prompt": prompt},
            return_source_documents=True
        )
        
        # Call QA chain with pre-retrieved documents
        context = "\n\n".join([doc.page_content for doc in docs])
        response = llm.predict(
            prompt.format(context=context, question=question)
        )
        
        return {
            "answer": response,
            "source_documents": docs
        }

# def main():
#     """Main function to run the maternal health pipeline."""
#     import argparse
    
#     parser = argparse.ArgumentParser(description="Process maternal health documents")
#     parser.add_argument("--input", required=True, help="Input file or directory")
#     parser.add_argument("--output", default="maternal_health_db", help="Output directory for vector store")
#     parser.add_argument("--embedding", default="google/muril-base-cased", help="Embedding model name")
#     parser.add_argument("--no-synthetic", action="store_true", help="Skip synthetic chunk creation")
#     parser.add_argument("--chunk-size", type=int, default=2500, help="Chunk size")
#     parser.add_argument("--no-spacy", action="store_true", help="Disable SpaCy (use simpler chunking)")
    
#     args = parser.parse_args()
    
#     # Initialize pipeline
    # pipeline = MaternalHealthPipeline(
    #     chunk_size=args.chunk_size,
    #     embedding_model_name=args.embedding,
    #     db_directory=args.output,
    #     use_spacy=not args.no_spacy
    # )
    
#     # Process documents
#     if os.path.isdir(args.input):
#         documents = pipeline.process_directory(args.input)
#     else:
#         documents = pipeline.process_document(args.input)
    
#     # Create vector store
#     if documents:
#         pipeline.create_vector_store(documents, include_synthetic=not args.no_synthetic)
#         logger.info(f"Successfully processed {len(documents)} chunks and created vector store in {args.output}")
#     else:
#         logger.error("No documents were processed.")

# if __name__ == "__main__":
#     main()

In [10]:
pipeline = MaternalHealthPipeline(
        chunk_size=3000,
        db_directory="/data/user_data/vidhij2/medical_db/rag_2"
    )

2025-06-11 02:00:25,725 - INFO - Initializing embeddings model: google/muril-base-cased
2025-06-11 02:00:25,729 - INFO - Load pretrained SentenceTransformer: intfloat/multilingual-e5-large-instruct
2025-06-11 02:00:28,344 - INFO - Loaded SpaCy model: en_core_web_md
2025-06-11 02:00:28,345 - INFO - Loading existing vector store from /data/user_data/vidhij2/medical_db/rag_2/index.faiss


In [11]:
documents = pipeline.process_directory("/data/user_data/vidhij2/medical/html")
    

2025-06-11 02:00:33,560 - INFO - Processing 83 files in /data/user_data/vidhij2/medical/html
Processing files:   0%|          | 0/83 [00:00<?, ?it/s]2025-06-11 02:00:33,561 - INFO - Processing /data/user_data/vidhij2/medical/html/Motherhood_Hospitals.html...
2025-06-11 02:00:33,683 - INFO - Splitting Motherhood_Hospitals.html into sections...
2025-06-11 02:00:33,687 - INFO - Found 11 sections
Processing sections: 100%|██████████| 11/11 [00:04<00:00,  2.47it/s]
2025-06-11 02:00:38,139 - INFO - Created 11 chunks from Motherhood_Hospitals.html
Processing files:   1%|          | 1/83 [00:04<06:15,  4.58s/it]2025-06-11 02:00:38,140 - INFO - Processing /data/user_data/vidhij2/medical/html/EuroKids_India.html...
2025-06-11 02:00:38,170 - INFO - Splitting EuroKids_India.html into sections...
2025-06-11 02:00:38,171 - INFO - Found 1 sections
Processing sections: 100%|██████████| 1/1 [00:00<00:00,  2.09it/s]
2025-06-11 02:00:38,651 - INFO - Created 4 chunks from EuroKids_India.html
Processing fi

In [5]:
document_2 = pipeline.process_directory("/data/user_data/vidhij2/medical/pdfs")

2025-06-11 01:41:34,510 - INFO - Processing 131 files in /data/user_data/vidhij2/medical/pdfs
Processing files:   0%|          | 0/131 [00:00<?, ?it/s]2025-06-11 01:41:34,512 - INFO - Processing /data/user_data/vidhij2/medical/pdfs/Guideline_for_Stammering_and_Stuttering.pdf...
2025-06-11 01:41:35,448 - INFO - Splitting Guideline_for_Stammering_and_Stuttering.pdf into sections...
2025-06-11 01:41:35,450 - INFO - Found 1 sections
Processing sections: 100%|██████████| 1/1 [00:00<00:00,  3.15it/s]
2025-06-11 01:41:35,769 - INFO - Created 3 chunks from Guideline_for_Stammering_and_Stuttering.pdf
Processing files:   1%|          | 1/131 [00:01<02:43,  1.26s/it]2025-06-11 01:41:35,770 - INFO - Processing /data/user_data/vidhij2/medical/pdfs/Guideline_for_Vitamins_and_Minerals_for_My_Child.pdf...
2025-06-11 01:41:36,100 - INFO - Splitting Guideline_for_Vitamins_and_Minerals_for_My_Child.pdf into sections...
2025-06-11 01:41:36,101 - INFO - Found 1 sections
Processing sections: 100%|██████████

In [12]:
documents.extend(document_2)

In [15]:
len(documents)

4970

In [14]:
pipeline.create_vector_store(documents)

2025-06-11 02:37:24,147 - INFO - Creating synthetic chunks for improved retrieval...


2025-06-11 02:37:24,220 - INFO - Created 72 synthetic chunks
2025-06-11 02:37:24,221 - INFO - Creating vector store with 5042 documents...
2025-06-11 02:38:22,778 - INFO - Saving vector store to /data/user_data/vidhij2/medical_db/rag_2


Testing the vector store

In [16]:
def answer_maternal_health_query(pipeline, query):
    """
    Answer a maternal health query using the pipeline.
    
    Args:
        pipeline: Initialized MaternalHealthPipeline
        query: Question from a pregnant woman
        
    Returns:
        Answer with source information
    """
    print(f"\nProcessing query: {query}")
    
    # Detect context from query
    context = pipeline.detect_query_context(query)
    stage = context["stage"] or "general"
    concerns = ", ".join(context["concerns"]) if context["concerns"] else "none detected"
    print(f"Detected stage: {stage}, concerns: {concerns}")
    
    # Retrieve relevant documents
    print("Retrieving relevant information...")
    docs = pipeline.retrieve_for_query(query)
    
    # Print source information
    print(f"Found {len(docs)} relevant chunks:")
    for i, doc in enumerate(docs[:3]):  # Show top 3 sources
        source = doc.metadata.get("title", "Unknown document")
        content_type = doc.metadata.get("content_type", "general")
        print(f"  Source {i+1}: {source} ({content_type})")
    
    # Generate answer
    print("Generating answer...")
    response = pipeline.answer_question(query, model_name="gpt-4")
    
    return response

def demo_queries():
    """Run demo queries to test the pipeline."""
    return [
        "I'm 8 weeks pregnant and having morning sickness. Is this normal and what can I do?",
        "What foods should I avoid during my pregnancy?",
        "I'm in my third trimester and having trouble sleeping. Any advice?",
        "I'm feeling my baby move less today. Should I be concerned?", 
        "What are the warning signs I should look out for after giving birth?",
        "I'm 6 months pregnant and having headaches. What could be causing this?"
    ]

def interactive_mode(pipeline):
    """Run in interactive mode to answer user questions."""
    print("\n==== Maternal Health Assistant ====")
    print("Type your pregnancy or postpartum questions, or 'quit' to exit.")
    
    while True:
        query = input("\nQuestion: ").strip()
        
        if query.lower() in ['quit', 'exit', 'q']:
            break
            
        if not query:
            continue
            
        response = answer_maternal_health_query(pipeline, query)
        print("\nAnswer:")
        print(response["answer"])




In [17]:
# demo
queries = demo_queries()
for query in queries:
    response = answer_maternal_health_query(pipeline, query)
    print("\nAnswer:")
    
    print(response["answer"])
    input("\nPress Enter for next query...")

2025-06-11 02:40:36,258 - INFO - Query context: {'stage': 'first_trimester', 'concerns': ['nausea'], 'detected_topics': ['normal_variations']}
2025-06-11 02:40:36,282 - INFO - Query context: {'stage': 'first_trimester', 'concerns': ['nausea'], 'detected_topics': ['normal_variations']}



Processing query: I'm 8 weeks pregnant and having morning sickness. Is this normal and what can I do?
Detected stage: first_trimester, concerns: nausea
Retrieving relevant information...
Found 4 relevant chunks:
  Source 1: Virtual Tour




Take A Virtual Tour


Virtual Tour






Care (information)
  Source 2: Virtual Tour




Take A Virtual Tour


Virtual Tour






Care (recommendation)
  Source 3: Continue




Skip to main content


















Health (information)
Generating answer...


/tmp/ipykernel_4038970/277868169.py:1564: LangChainDeprecationWarning: The method `BaseChatModel.predict` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llm.predict(
2025-06-11 02:40:45,212 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"



Answer:
Yes, experiencing morning sickness at 8 weeks of pregnancy is quite normal. Morning sickness, also known as pregnancy-related nausea and vomiting, is common during early pregnancy. Despite its name, it can occur at any time of the day.

There are several strategies you can try to manage your morning sickness:

1. Dietary Changes: Consuming ginger in various forms, such as ginger tea, ginger candies, or ginger capsules, may help reduce nausea. Try to identify and avoid foods or smells that trigger or worsen your symptoms.

2. Hydration: It's important to stay hydrated during pregnancy. Aim for 8 to 12 cups of water a day. Not drinking enough fluids can lead to dehydration, which can make nausea worse.

3. Vitamin B6: In some cases, healthcare providers may recommend vitamin B6 supplements to manage morning sickness.

4. Medications: If your symptoms are severe and self-care measures are not effective, your healthcare provider may prescribe anti-nausea medications that are safe 

In [ ]:
# interactive
interactive_mode(pipeline)

loading vector store

In [3]:
# embeddings = HuggingFaceEmbeddings(
#     model_name="google/muril-base-cased",
#     model_kwargs={'device': 'cpu'},
#     encode_kwargs={'normalize_embeddings': True}
# )
embeddings = E5Embeddings(device="cuda")
vector_store = FAISS.load_local(
    folder_path="/data/user_data/vidhij2/medical_db/rag_2",
    embeddings=embeddings,  # This must be the properly initialized embeddings
    allow_dangerous_deserialization=True
)

/home/vidhij2/.local/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-06-14 02:25:29.775705: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-06-14 02:25:29.860881: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749882329.900535  687616 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749882329.912225  687616 cuda_blas.cc:1418] Unable to 

In [18]:
vector_store = FAISS.load_local(folder_path="/data/user_data/vidhij2/medical_db/rag_2", embeddings="google/muril-base-cased", allow_dangerous_deserialization=True)

2025-06-11 02:39:16,981 - WARNING - `embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.


In [6]:
vector_store.index.ntotal

5042

Enhanced RAG system for maternal health with multilingual support,
    stage-aware retrieval, and cross-encoder reranking. This is the answering thing after vector store is created

In [5]:
from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA, LLMChain
from langchain.prompts import PromptTemplate
from typing import List, Dict, Any, Optional, Tuple, Union
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.schema import Document
from langchain.llms import HuggingFacePipeline
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification
import torch
import numpy as np
from sentence_transformers import CrossEncoder
import re
from functools import lru_cache

class EnhancedMedicalRAG:
    """
    Enhanced RAG system for maternal health with multilingual support,
    stage-aware retrieval, and cross-encoder reranking.
    """

    def __init__(
        self,
        vector_store: FAISS,
        model,
        tokenizer,
        temperature: float = 0.25,
        reranker_model: str = "cross-encoder/ms-marco-MiniLM-L-6-v2",
        use_reranking: bool = True
    ):
        # Set up text generation pipeline
        
        self.vector_store = vector_store

        if model == "gpt-4-turbo":
            self.llm = ChatOpenAI(
                model=model,
                temperature=0.1  # Low temperature for factual responses
            )
        else:
            pipe = pipeline(
            "text-generation",
            model=model,
            tokenizer=tokenizer,
            max_new_tokens=512,
            device_map="auto",
            temperature=temperature,  # Lower temperature for medical accuracy
            top_p=0.95,
            batch_size=1
            )
            self.llm = HuggingFacePipeline(
                pipeline=pipe
            )
        self.use_reranking = use_reranking
        
        # Initialize reranker model if enabled
        if use_reranking:
            try:
                self.reranker = CrossEncoder(reranker_model)
                print(f"Initialized reranker model: {reranker_model}")
            except Exception as e:
                print(f"Failed to load reranker model: {e}")
                self.use_reranking = False
        
        # Language detection model - simple regex patterns for quick language detection
        self.language_patterns = {
            'hindi': re.compile(r'[\u0900-\u097F]'),   # Hindi Unicode range
            'assamese': re.compile(r'[\u0980-\u09FF]'),  # Bengali/Assamese Unicode range
            'english': re.compile(r'[a-zA-Z]')
        }
        
        # Create pregnancy stage patterns for query understanding
        self.stage_patterns = {
            'week': re.compile(r'(?:week|weeks)\s+(\d+(?:-\d+)?)|(\d+)(?:st|nd|rd|th)?\s+week', re.IGNORECASE),
            'month': re.compile(r'(?:month|months)\s+(\d+(?:-\d+)?)|(\d+)(?:st|nd|rd|th)?\s+month', re.IGNORECASE),
            'trimester': re.compile(r'(first|second|third)\s+trimester', re.IGNORECASE),
            'stage': re.compile(r'(preconception|prenatal|antenatal|perinatal|intrapartum|postpartum|postnatal)', re.IGNORECASE)
        }
        
        # Medical concern patterns for better filtering
        self.concern_patterns = {
            'pain': re.compile(r'pain|ache|hurt|sore|discomfort', re.IGNORECASE),
            'bleeding': re.compile(r'bleed|blood|spotting|hemorrhage', re.IGNORECASE),
            'movement': re.compile(r'movement|kick|moving|motion|active', re.IGNORECASE),
            'nutrition': re.compile(r'food|eat|diet|nutrition|hungry|appetite', re.IGNORECASE),
            'sleep': re.compile(r'sleep|insomnia|tired|fatigue|rest', re.IGNORECASE),
            'emotional': re.compile(r'depress|anxiety|worry|stress|mood|emotion', re.IGNORECASE)
        }

        # Define prompt templates
        self.rag_prompt = PromptTemplate(
            template="""You are a supportive maternal health assistant providing information to pregnant women and new mothers.
Use the following retrieved information to answer the question.
Response Guidelines:
1. Answer ONLY using the provided context. Do not speculate or generate information not found in the retrieved data. If the provided context doesn't fully answer the question, state the limitations. Example: “I don’t have enough information to answer that accurately.”


3. Be reassuring but honest. Do not minimize legitimate concerns. If the topic is serious, clearly explain when to seek medical help.


4. Use clear, non-technical language. Avoid jargon and explain complex terms simply.

5. Warning signs: When discussing symptoms or warning signs, clearly distinguish between normal variations and symptoms that require medical attention. List these separately if appropriate.

6. If you detect any signs of a possible emergency (such as heavy bleeding, severe pain, loss of consciousness, seizures, absent fetal movement, severe swelling, difficulty breathing, fever with rash, premature rupture of membranes, or vision changes):

Do NOT give any advice.

Respond ONLY with:
"IMPORTANT: This appears to be an emergency situation that requires immediate medical attention.

This chatbot cannot provide emergency medical advice. Please:
1. Contact your healthcare provider immediately OR
2. Go to the nearest emergency department OR
3. Call your local emergency number for immediate assistance.

Do not wait or delay seeking professional care."


7. Do not provide: Any specific drug brand or prescription recommendations; Any information related to fetal gender detection or selection; Any instructions or advice for suicidal/self-harm situations. Instead, respond only with the relevant crisis hotline (India: 9152987821, US: 988, UK: 0800 689 5652).

8. Do not answer off topic questions, ie: questions which are not related to  pregnancy, postpartum, or maternal/newborn wellness at all. (if off-topic, say: “I’m sorry, I can only help with pregnancy and maternal-health questions.”),

9. Respond in the same language as the original question, if the language is hindi written in english, do the same. 

Context:
 {context}
Question:
 {question}
Answer:
""",

            
# Context information from trusted maternal health guidelines:
# {context}

# User Question: {question}

# Instructions:
# 1. Respond based only on the provided context. If the information isn't in the context, say "I don't have enough information to answer this question based on the provided guidelines."
# 2. Be reassuring but accurate - don't minimize legitimate health concerns.
# 3. If discussing warning signs, clearly distinguish between normal variations and concerning symptoms.
# 4. Remind users to consult healthcare providers for personal medical advice when appropriate.
# 5. Use clear, non-technical language that is easy to understand.
# 6. Respond in the same language as the original question, if the language is hindi written in english, do the same. 

# Answer:""",
            input_variables=["context", "question"]
        )

        # Create LLM chain
        self.qa_prompt_chain = LLMChain(
            llm=self.llm,
            prompt=self.rag_prompt
        )

    @lru_cache(maxsize=100)
    def translate_query_to_english(self, query: str) -> str:
        """Translate query to English if needed, with caching for efficiency."""
        # Skip translation if query already contains mostly English
        
            
        translation_prompt = PromptTemplate(
            template="""Translate the following maternal health query to English, preserving all medical terms and meaning:

Query: {query}

Translation:""",
            input_variables=["query"]
        )
        
        try:
            llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
            response = llm(translation_prompt.format_prompt(query=query).to_messages())
            return response.content.strip()
        except Exception as e:
            print(f"Translation failed: {e}")
            return query  # Return original query if translation fails

    def _detect_language(self, text: str) -> str:
        """Detect the primary language of the text."""
        counts = {
            lang: len(pattern.findall(text)) 
            for lang, pattern in self.language_patterns.items()
        }
        
        # If mixed, determine based on ratio
        total = sum(counts.values())
        if total == 0:
            return 'english'  # Default to English if no matches
            
        # Get language with highest ratio
        lang_ratios = {lang: count/total for lang, count in counts.items()}
        primary_lang = max(lang_ratios.items(), key=lambda x: x[1])[0]
        
        # Check if it's mixed (no language has > 70% dominance)
        if lang_ratios[primary_lang] < 0.7:
            return 'mixed'
            
        return primary_lang

    def detect_pregnancy_stage(self, query: str) -> Optional[str]:
        """Extract pregnancy stage information from the query."""
        query_lower = query.lower()
        
        # Map from detected stage references to standardized stages
        stage_mapping = {
            'first_trimester': ['first trimester', 'early pregnancy', '1st trimester'],
            'second_trimester': ['second trimester', 'mid pregnancy', '2nd trimester'],
            'third_trimester': ['third trimester', 'late pregnancy', '3rd trimester'],
            'labor_delivery': ['labor', 'delivery', 'birth', 'intrapartum'],
            'postpartum': ['postpartum', 'after birth', 'post delivery', 'postnatal']
        }
        
        # Check for direct stage mentions
        for stage, keywords in stage_mapping.items():
            if any(keyword in query_lower for keyword in keywords):
                return stage
                
        # Check for week mentions
        for match in self.stage_patterns['week'].finditer(query_lower):
            week_str = match.group(1) or match.group(2)
            if week_str:
                try:
                    week = int(week_str.split('-')[0])  # Take first number if range
                    if 1 <= week <= 12:
                        return 'first_trimester'
                    elif 13 <= week <= 26:
                        return 'second_trimester'
                    elif 27 <= week <= 42:
                        return 'third_trimester'
                except ValueError:
                    pass
        
        # Check for month mentions
        for match in self.stage_patterns['month'].finditer(query_lower):
            month_str = match.group(1) or match.group(2)
            if month_str:
                try:
                    month = int(month_str.split('-')[0])  # Take first number if range
                    if 1 <= month <= 3:
                        return 'first_trimester'
                    elif 4 <= month <= 6:
                        return 'second_trimester'
                    elif 7 <= month <= 10:
                        return 'third_trimester'
                except ValueError:
                    pass
                    
        return None  # No stage detected

    def detect_concerns(self, query: str) -> List[str]:
        """Detect maternal health concerns from query."""
        query_lower = query.lower()
        concerns = []
        
        for concern, pattern in self.concern_patterns.items():
            if pattern.search(query_lower):
                concerns.append(concern)
                
        return concerns

    def hybrid_retrieve(self, query: str, k: int = 10) -> List[Document]:
        """Retrieve documents using both dense and metadata-based approaches."""
        stage = self.detect_pregnancy_stage(query)
        concerns = self.detect_concerns(query)
        
        # Default retriever - dense vector similarity
        search_kwargs = {"k": k}
        
        # Add stage filter if detected
        if stage:
            # FIXED: Use a filter function that works with metadata dictionaries
            stage_filter = lambda metadata: (
                # Match documents with the same stage or no specific stage
                stage in metadata.get('pregnancy_stages', {}).get('standard_stages', [])
                if isinstance(metadata.get('pregnancy_stages', {}), dict) else False
            ) or (
                not (metadata.get('pregnancy_stages', {}).get('standard_stages', [])
                    if isinstance(metadata.get('pregnancy_stages', {}), dict) else [])
            )
            search_kwargs["filter"] = stage_filter
        
        # First retrieve based on vector similarity
        initial_docs = self.vector_store.as_retriever(search_kwargs=search_kwargs).get_relevant_documents(query)
        
        # Check if concerns are addressed in top results
        if concerns:
            concern_addressed = False
            for doc in initial_docs[:3]:  # Check top 3 docs
                doc_concerns = doc.metadata.get('patient_concerns', [])
                if any(concern in doc_concerns for concern in concerns):
                    concern_addressed = True
                    break
                    
            # If concerns not addressed, get concern-specific docs
            if not concern_addressed:
                # FIXED: Use a filter function that works with metadata dictionaries
                concern_filter = lambda metadata: any(
                    concern in metadata.get("patient_concerns", []) 
                    for concern in concerns
                )
                
                # Create a temporary retriever with this filter
                concern_kwargs = search_kwargs.copy()
                concern_kwargs["filter"] = concern_filter
                concern_kwargs["k"] = 3  # Just get a few concern-specific docs
                
                concern_docs = self.vector_store.as_retriever(
                    search_kwargs=concern_kwargs
                ).get_relevant_documents(query)
                
                # Add unique concern docs (avoid duplicates)
                existing_ids = {doc.metadata.get('chunk_id', '') for doc in initial_docs}
                for doc in concern_docs:
                    if doc.metadata.get('chunk_id', '') not in existing_ids:
                        initial_docs.append(doc)
                        
        return initial_docs

    def rerank_documents(self, query: str, docs: List[Document], top_k: int = 7) -> List[Document]:
        """Rerank documents using a cross-encoder if available."""
        if not self.use_reranking or not docs:
            return docs[:top_k]
            
        try:
            # Create query-document pairs
            query_doc_pairs = [(query, doc.page_content) for doc in docs]
            
            # Get scores from reranker
            scores = self.reranker.predict(query_doc_pairs)
            
            # Sort by score
            doc_score_pairs = sorted(zip(docs, scores), key=lambda x: x[1], reverse=True)
            reranked_docs = [doc for doc, _ in doc_score_pairs]
            
            return reranked_docs[:top_k]
        except Exception as e:
            print(f"Reranking failed: {e}")
            return docs[:top_k]  # Return original ordering if reranking fails

    def query(self, question: str, translate: bool = True) -> Dict[str, Any]:
        """Query the enhanced RAG system with multilingual support and reranking."""
        original_question = question
        
        # Detect language
        detected_language = self._detect_language(question)
        print(f"Detected language: {detected_language}")
        # Translate if needed
        if translate :
            translated_question = self.translate_query_to_english(question)
        else:
            translated_question = question
        print(f"Translated question: {translated_question}")
        # Retrieve documents using hybrid approach
        retrieved_docs = self.hybrid_retrieve(translated_question)
        
        # Rerank if enabled
        if self.use_reranking:
            reranked_docs = self.rerank_documents(translated_question, retrieved_docs)
        else:
            reranked_docs = retrieved_docs[:7]  # Limit to top 7 if not reranking
            
        # Join contexts
        context = "\n\n".join(doc.page_content for doc in reranked_docs)
        # print(f"Context: {context}")
        # Generate answer with original question
        answer = self.qa_prompt_chain.run({
            "context": context,
            "question": original_question
        })
        
        # Clean up answer
        answer_start = answer.lower().find("answer:")
        if answer_start != -1:
            answer = answer[answer_start + 7:].strip()  # +7 to skip "answer:"
            
        # Prepare source information
        sources = [
            {
                "content": doc.page_content[:300] + "..." if len(doc.page_content) > 300 else doc.page_content,
                "metadata": {
                    k: v for k, v in doc.metadata.items() 
                    if k in ['title', 'source', 'pregnancy_stages', 'content_type']
                }
            }
            for doc in reranked_docs
        ]
        
        # Include detected stage and concerns for debugging/monitoring
        detected_info = {
            "language": detected_language,
            "stage": self.detect_pregnancy_stage(question),
            "concerns": self.detect_concerns(question)
        }

        return {
            "question_original": original_question,
            "question_translated": translated_question,
            "answer": answer,
            "sources": sources,
            "detected_info": detected_info
        }

    def query_without_rag(self, question: str, translate: bool = True) -> Dict[str, Any]:
        """Query without retrieval for comparison purposes."""
        original_question = question
        detected_language = self._detect_language(question)
        
        if translate and detected_language != 'english':
            translated_question = self.translate_query_to_english(question)
        else:
            translated_question = question

        # Define a prompt that doesn't use context
        direct_prompt = PromptTemplate(
            template="""You are a supportive maternal health assistant providing information to pregnant women and new mothers.
Response Guidelines:
1. Answer based on your knowledge.
3. Be reassuring but honest. Do not minimize legitimate concerns. If the topic is serious, clearly explain when to seek medical help.


4. Use clear, non-technical language. Avoid jargon and explain complex terms simply.

5. Warning signs: When discussing symptoms or warning signs, clearly distinguish between normal variations and symptoms that require medical attention. List these separately if appropriate.

6. If you detect any signs of a possible emergency, answer accordingly:

7. Do not provide: Any specific drug brand or prescription recommendations; Any information related to fetal gender detection or selection; Any instructions or advice for suicidal/self-harm situations. Instead, respond only with the relevant crisis hotline (India: 9152987821, US: 988, UK: 0800 689 5652).

8. Do not answer off topic questions, ie: questions which are not related to  pregnancy, postpartum, or maternal/newborn wellness at all. (if off-topic, say: “I’m sorry, I can only help with pregnancy and maternal-health questions.”),

9. Respond in the same language as the original question, if the language is hindi written in english, do the same. 

Question:
 {question}
Answer:

""",
            input_variables=["question"]
        )

        qa_direct_chain = LLMChain(
            llm=self.llm,
            prompt=direct_prompt
        )

        answer = qa_direct_chain.run({
            "question": original_question
        })
        
        # Clean up answer
        answer_start = answer.lower().find("answer:")
        if answer_start != -1:
            answer = answer[answer_start + 7:].strip()
            
        # Include detected info for consistency
        detected_info = {
            "language": detected_language,
            "stage": self.detect_pregnancy_stage(question),
            "concerns": self.detect_concerns(question)
        }
            
        return {
            "question_original": original_question,
            "question_translated": translated_question,
            "answer": answer,
            "sources": [],  # No sources in non-RAG mode
            "detected_info": detected_info
        }

# Example usage code:
"""
# Initialize with your chunked vector store
from langchain.vectorstores import FAISS
from transformers import AutoModelForCausalLM, AutoTokenizer

# Load your vector store
embeddings = HuggingFaceEmbeddings(model_name="google/muril-base-cased")
vector_store = FAISS.load_local(
    folder_path="maternal_health_db",
    embeddings=embeddings,
    allow_dangerous_deserialization=True
)

# Load your LLM (e.g., Llama-3-8B or other appropriate model)
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3-8B-Instruct")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3-8B-Instruct")

# Initialize the enhanced RAG system
rag_system = EnhancedMedicalRAG(
    vector_store=vector_store,
    model=model,
    tokenizer=tokenizer,
    use_reranking=True  # Set to False if you don't want to use reranking
)

# Example query in Hindi
hindi_query = "मेरे गर्भ में 7 महीने का बच्चा है और वह कम हिल रहा है। क्या यह चिंता का विषय है?"
result = rag_system.query(hindi_query)
print(result["answer"])

# Example query in Assamese
assamese_query = "মোৰ 6 মাহৰ গৰ্ভাৱস্থা চলি আছে আৰু মোৰ বাৰে বাৰে পিঠি বিষাই থাকে। ই সাধাৰণ নেকি?"
result = rag_system.query(assamese_query)
print(result["answer"])

# Example query in English
english_query = "I'm 30 weeks pregnant and experiencing headaches. What could be causing this?"
result = rag_system.query(english_query)
print(result["answer"])
"""

'\n# Initialize with your chunked vector store\nfrom langchain.vectorstores import FAISS\nfrom transformers import AutoModelForCausalLM, AutoTokenizer\n\n# Load your vector store\nembeddings = HuggingFaceEmbeddings(model_name="google/muril-base-cased")\nvector_store = FAISS.load_local(\n    folder_path="maternal_health_db",\n    embeddings=embeddings,\n    allow_dangerous_deserialization=True\n)\n\n# Load your LLM (e.g., Llama-3-8B or other appropriate model)\ntokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3-8B-Instruct")\nmodel = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3-8B-Instruct")\n\n# Initialize the enhanced RAG system\nrag_system = EnhancedMedicalRAG(\n    vector_store=vector_store,\n    model=model,\n    tokenizer=tokenizer,\n    use_reranking=True  # Set to False if you don\'t want to use reranking\n)\n\n# Example query in Hindi\nhindi_query = "मेरे गर्भ में 7 महीने का बच्चा है और वह कम हिल रहा है। क्या यह चिंता का विषय है?"\nresult = rag_system.que

In [7]:
from langchain.vectorstores import FAISS
from transformers import AutoModelForCausalLM, AutoTokenizer


In [8]:
tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-3-8B-Instruct")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-3-8B-Instruct")

OSError: meta-llama/Llama-3-8B-Instruct is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`

In [6]:
rag_system = EnhancedMedicalRAG(
    vector_store=vector_store,
    model='gpt-4-turbo',
    tokenizer= None,
    use_reranking=True  # Set to False if you don't want to use reranking
)

/tmp/ipykernel_2086988/4158433042.py:36: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  self.llm = ChatOpenAI(
2025-06-14 01:14:03,382 - INFO - Use pytorch device: cuda


Initialized reranker model: cross-encoder/ms-marco-MiniLM-L-6-v2


/tmp/ipykernel_2086988/4158433042.py:152: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  self.qa_prompt_chain = LLMChain(


In [17]:
hindi_query = "Pragnancy me mango 🥭 kitna mntra me kha skti hu"
result = rag_system.query(hindi_query)
print(result["answer"])

Detected language: english


2025-06-12 00:13:28,860 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Translated question: How much mango 🥭 can I eat during pregnancy?


Batches: 100%|██████████| 1/1 [00:00<00:00, 112.98it/s]
2025-06-12 00:13:30,109 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


I’m sorry, I can only help with pregnancy and maternal-health questions.


In [18]:
hindi_query = "मेरे गर्भ में 7 महीने का बच्चा है और वह कम हिल रहा है। क्या यह चिंता का विषय है?"
result = rag_system.query(hindi_query)
print(result["answer"])

Detected language: hindi


2025-06-12 00:13:37,097 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Translated question: Query: I am 7 months pregnant and the baby is moving less. Is this a cause for concern?


Batches: 100%|██████████| 1/1 [00:00<00:00, 83.02it/s]
2025-06-12 00:13:46,042 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


हां, यह चिंता का विषय हो सकता है। यदि आपका बच्चा सामान्य से कम हिल रहा है, तो यह जरूरी है कि आप तुरंत अपने स्वास्थ्य सेवा प्रदाता से संपर्क करें। नवजात शिशुओं को आमतौर पर अपने हाथ या पैर हिलाने या सिर घुमाने की क्रिया कई बार एक मिनट में करनी चाहिए। कम हरकत एक चेतावनी संकेत हो सकता है और इसे गंभीरता से लेना चाहिए।

कृपया इसे अनदेखा न करें और जल्द से जल्द चिकित्सा सहायता प्राप्त करें।


In [6]:
queries= ["My baby weight is 452 gram in 22weeks..is this normal",
"Sometimes I don’t feel the baby’s move ..is it okay or something else?",
"I have seen prickly heat and itching on some part of my body... What should I do?",

"Pragnancy me mango 🥭 kitna mntra me kha skti hu",
"Bacche ko Tej Sardi ho to kaise usko maintain Karen",
"Kya pregnancy m bleeding hoti h kya yeh normal h",
"mujhe thyroid h to kya isse bhi breast milk pe asar hota h",
"2.5 months hogae h pregnancy ko but vomet ya kch ni hora mjhe", 

"जयदा टाइम तक बैठने से चक्कर आना खड़ा होने से पेट दर्द होने लगता है",
"मेरे को कौन सी टेबलेट खानी चाहिए अभी सट्टा मा लग चुका है",
"सर्दी -जुकाम  हो रहा  है।इसका बच्चे पर तो कोई प्रभाव  नहीं  होगा ।",
"I am 33 weeks pregnant and suddenly gushing bright‑red blood with sharp cramps.",
"Is it safe to take 500 mg amoxicillin three times a day for my UTI?",
"I’m feeling constant chest pain and can barely catch my breath at 29 weeks."



]

In [13]:
all_results = []
for q in queries:
    result = rag_system.query(q)
    
    # Print to console (optional)
    # print(result["answer"])
   

    # Add result to the list
    all_results.append(result)

Detected language: english


/tmp/ipykernel_1822105/2194079539.py:174: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llm(translation_prompt.format_prompt(query=query).to_messages())
2025-06-12 02:13:17,179 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
/tmp/ipykernel_1822105/2194079539.py:285: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  initial_docs = self.vector_store.as_retriever(search_kwargs=search_kwargs).get_relevant_documents(query)


Translated question: My baby weighs 452 grams at 22 weeks..is this normal


Batches: 100%|██████████| 1/1 [00:00<00:00, 61.89it/s]
/tmp/ipykernel_1822105/2194079539.py:368: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  answer = self.qa_prompt_chain.run({
2025-06-12 02:13:20,343 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Detected language: english


...

Batches: 100%|██████████| 1/1 [00:00<00:00, 96.09it/s]
2025-06-12 02:14:42,641 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Detected language: english


2025-06-12 02:14:43,298 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Translated question: I am experiencing persistent chest pain and have difficulty breathing at 29 weeks of pregnancy.


Batches: 100%|██████████| 1/1 [00:00<00:00, 121.87it/s]
2025-06-12 02:14:45,322 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


In [24]:
all_results

[{'question_original': 'My baby weight is 452 gram in 22weeks..is this normal',
  'question_translated': 'My baby weighs 452 grams at 22 weeks..is this normal',
  'answer': "I don’t have enough information to answer that accurately. For specific concerns about your baby's weight and growth, it's best to consult with your healthcare provider who can provide guidance based on your individual medical history and the specifics of your pregnancy.",
  'sources': [{'content': '1.  We will measure your weight and blood pressure and will look for any signs of de ﬁ ciencies or disorders; since this is the last month of \npregnancy, keep all your medical records and some cash handy\n2.  As your baby is gaining weight you should include energy dense food items like cereals (whea...',
    'metadata': {'title': 'Maternal Nutrition For Safe Motherhood. Messages By Gestational Month',
     'pregnancy_stages': {'weeks': [],
      'months': [],
      'trimesters': [],
      'named_stages': [],
      'st

In [18]:
import json

with open("test_docs/gpt-4-turbo.json", "w", encoding="utf-8") as f:
    json.dump(all_results, f, ensure_ascii=False, indent=4)

In [9]:
all_results = []
for q in queries:
    result = rag_system.query_without_rag(q)
    
    # Print to console (optional)
    print(result["answer"])
   

    # Add result to the list
    all_results.append(result)

/tmp/ipykernel_2086988/4158433042.py:449: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  answer = qa_direct_chain.run({
2025-06-14 01:14:35,237 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


At 22 weeks, the average fetal weight is typically around 430 grams, although this can vary. A weight of 452 grams at this stage is within a normal range and generally not a cause for concern. It's important to continue regular prenatal check-ups with your healthcare provider. They can monitor your baby's growth and development, ensuring everything is progressing well. If you have any concerns about your pregnancy or your baby's growth, don't hesitate to discuss them with your doctor. They are there to support you throughout your pregnancy journey.


2025-06-14 01:14:41,317 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


It's completely understandable to feel concerned if you notice that your baby isn't moving as much as usual. Here’s some information that might help you understand what might be happening:

1. **Normal Variations in Baby Movement:**
   - As your pregnancy progresses, the pattern of your baby's movements may change. Early in pregnancy, you might not feel every movement, and the types of movements may change as the baby grows.
   - Babies also have sleep cycles and might be less active during certain times of the day.

2. **When to Seek Medical Help:**
   - If you are past 28 weeks and notice a significant decrease in movements or if your baby is moving much less than usual (less than 10 movements in 2 hours), it is important to seek medical attention immediately.
   - Any sudden or drastic changes in the pattern of movements should be checked by your healthcare provider.

To monitor your baby’s movements, you might find it helpful to pick a quiet time each day to feel for movements. Lyi

2025-06-14 01:14:47,481 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


...

It's very important that you seek medical help immediately. Gushing bright-red blood and experiencing sharp cramps at 33 weeks pregnant could be signs of a serious condition, such as placental abruption or preterm labor. These conditions require urgent attention from healthcare professionals.

Please go to the nearest hospital or call your healthcare provider right away. It's essential to get checked by a medical professional to ensure the safety of both you and your baby.


2025-06-14 01:16:18,167 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


It's important to consult with your healthcare provider before starting any medication, including antibiotics like amoxicillin, especially during pregnancy. Amoxicillin is commonly prescribed for urinary tract infections (UTIs), and the dosage you mentioned is typical, but the safety and appropriateness of this medication for your specific situation should be confirmed by a doctor who knows your medical history and pregnancy details.

Taking antibiotics without proper medical guidance can lead to resistance, ineffective treatment, or unexpected side effects. If you're experiencing symptoms of a UTI, such as pain or burning during urination, frequent urination, or cloudy urine, it's important to talk to your healthcare provider. They can confirm the infection through appropriate tests and recommend the best treatment.

If you have already been prescribed this medication by your healthcare provider, it's important to take it exactly as prescribed and complete the full course, even if you

2025-06-14 01:16:21,207 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


It's important to take chest pain and difficulty breathing seriously, especially during pregnancy. These symptoms can sometimes indicate a more serious condition that needs immediate medical attention.

Please seek medical help right away by going to the nearest emergency room or calling your healthcare provider. It's essential to rule out any conditions that could be harmful to you or your baby.

While some breathlessness can be normal due to the growing uterus pressing against your diaphragm, constant chest pain and severe difficulty breathing are not typical and should be evaluated by a healthcare professional immediately.

Stay safe and please don't hesitate to seek help.


In [10]:
with open("test_docs/gpt-4-turbo_no_rag.json", "w", encoding="utf-8") as f:
    json.dump(all_results, f, ensure_ascii=False, indent=4)

In [7]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch
model_name =  "/data/models/huggingface/meta-llama/Meta-Llama-3-8B-chat"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
            model_name
        ).to("cuda")

Loading checkpoint shards:  75%|███████▌  | 3/4 [00:13<00:04,  4.60s/it]

: 

In [8]:
rag_system = EnhancedMedicalRAG(
    vector_store=vector_store,
    model=model,
    tokenizer=tokenizer_mixstral,
    use_reranking=True  # Set to False if you don't want to use reranking
)

Device set to use cuda:0
2025-06-14 00:48:22,724 - INFO - Use pytorch device: cuda


Initialized reranker model: cross-encoder/ms-marco-MiniLM-L-6-v2


In [10]:
all_results = []
for q in queries:
    result = rag_system.query(q)
    
    # Print to console (optional)
    print(result["answer"])
   

    # Add result to the list
    all_results.append(result)

Detected language: english


/tmp/ipykernel_2057070/2194079539.py:173: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0)
/tmp/ipykernel_2057070/2194079539.py:174: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llm(translation_prompt.format_prompt(query=query).to_messages())
2025-06-13 19:04:51,390 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
/tmp/ipykernel_2057070/2194079539.py:285: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0

Translated question: My baby weighs 452 grams at 22 weeks...is this normal?


Batches: 100%|██████████| 1/1 [00:00<00:00, 11.99it/s]
/tmp/ipykernel_2057070/2194079539.py:368: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  answer = self.qa_prompt_chain.run({
/home/vidhij2/.local/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:629: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.25` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/vidhij2/.local/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:634: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


"Thank you for your question. According to the information provided, your baby's weight is lower than the average for 22 weeks. Typically, a baby at 22 weeks should weigh around 450 to 500 grams. However, every baby is unique and may not follow the exact growth pattern. It is important to monitor your baby's growth regularly through ultrasounds and prenatal check-ups. If you have concerns about your baby's weight, please discuss them with your healthcare provider. They can provide you with accurate information based on your specific situation and may recommend additional monitoring or interventions if necessary. In the meantime, ensure you are eating a balanced diet with plenty of nutrient-dense foods to support your baby's growth. This includes cereals, pulses, milk and milk products, cooking oils, nuts and oilseeds, and jaggery. Also, consider including calcium-rich foods such as milk, curd, paneer, ragi, sesame seeds, bathua leaves, and methi leaves in your diet. If you need further

...

Yes, it is safe to take 500 mg amoxicillin three times a day for your UTI. However, make sure to complete the full course of medication as prescribed by your doctor, even if you start feeling better. Also, drink plenty of fluids to help flush out the bacteria from your urinary tract. If you have any concerns or if your symptoms do not improve after a few days, contact your healthcare provider.
Detected language: english


2025-06-13 19:11:46,862 - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"


Translated question: I am experiencing persistent chest pain and have difficulty breathing at 29 weeks of pregnancy.


Batches: 100%|██████████| 1/1 [00:00<00:00, 115.18it/s]
/home/vidhij2/.local/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:629: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.25` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/home/vidhij2/.local/lib/python3.9/site-packages/transformers/generation/configuration_utils.py:634: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


"IMPORTANT: This appears to be an emergency situation that requires immediate medical attention.

This chatbot cannot provide emergency medical advice. Please:
1. Contact your healthcare provider immediately OR
2. Go to the nearest emergency department OR
3. Call your local emergency number for immediate assistance.

Do not wait or delay seeking professional care."


In [28]:
pipe = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        max_length=512,       # Limit output size
        batch_size=1,         # Process one at a time
        device_map="auto"
    )

Device set to use cuda:0


In [11]:
all_results

[{'question_original': 'My baby weight is 452 gram in 22weeks..is this normal',
  'question_translated': 'My baby weighs 452 grams at 22 weeks..is this normal',
  'answer': '1.  As the 4 th month has started, we will have to repeat some of the \ntests; your physical examination and tests will include the following:\nMeasurements Tests Check for\nWeight Blood hemoglobin level Pallor (conjunctiva, tongue, oral \nmucosa and palms)\nPalpable goitre\nDental and skeletal fluorosis\nPedal edema or pufﬁ ness of face\nBlood pressure Urine sugar and albumin\n2.  As your baby is rapidly increasing in size now, eat at least 3 main \nmeals and 2 nutritious snacks to meet the increased nutrient \nrequirements to ensure proper growth and development of the baby\n\n• Jaundice is NOT NORMAL if\n- It has appeared at < 24 hrs. of  age\n- Palms and soles are stained\n\n\n5. Check the baby’s temperature every hour until normal.\n\n27\n/head2rightCase Study 8: (Ask the trainee to write down the answers to t

In [11]:
with open("test_docs/mixstral.json", "w", encoding="utf-8") as f:
    json.dump(all_results, f, ensure_ascii=False, indent=4)

In [1]:
all_results = []
for q in queries:
    result = rag_system.query_without_rag(q)
    
    # Print to console (optional)
    print(result["answer"])
   

    # Add result to the list
    all_results.append(result)

NameError: name 'queries' is not defined

In [13]:
with open("test_docs/llama.json", "w", encoding="utf-8") as f:
    json.dump(all_results, f, ensure_ascii=False, indent=4)

In [2]:
from transformers import BitsAndBytesConfig
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch
model_name = "/data/models/huggingface/mistralai/Mixtral-8x7B-Instruct-v0.1"

# Enable 8-bit quantization to reduce VRAM usage

# bnb_config = BitsAndBytesConfig(
#     load_in_8bit=True,
#     bnb_8bit_compute_dtype="float16"
# )  # Use load_in_4bit=True if needed

# Load tokenizer
tokenizer_mixstral = AutoTokenizer.from_pretrained(model_name)

# Load model in optimized mode
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16, 
    device_map="auto",
    trust_remote_code=True
)

print("✅ Mixtral loaded successfully in 8-bit mode.")

Loading checkpoint shards: 100%|██████████| 19/19 [13:02<00:00, 41.16s/it]
Some parameters are on the meta device because they were offloaded to the cpu.


✅ Mixtral loaded successfully in 8-bit mode.


In [18]:
python -m bitsandbytes

SyntaxError: invalid syntax (3988573585.py, line 1)

In [20]:
import torch
print(f"CUDA available: {torch.cuda.is_available()}")
print(f"CUDA device count: {torch.cuda.device_count()}")
if torch.cuda.is_available():
    print(f"CUDA device name: {torch.cuda.get_device_name(0)}")
    print(f"CUDA version: {torch.version.cuda}")

CUDA available: True
CUDA device count: 2
CUDA device name: NVIDIA RTX A6000
CUDA version: 12.4


In [33]:
pipe = pipeline(
                "text-generation",
                model=model,  # This should be your model object
                tokenizer=tokenizer,
                max_new_tokens=512,
                device_map="auto",
                temperature=0.25,
                top_p=0.95,
                batch_size=1  # Process one at a time to avoid CUDA errors
            )

Device set to use cuda:0
